In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

In [ ]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201228'
endDate = '20210108'

readPath = r'L:\orderLog\data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
try:
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r1, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])

try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
taking = orderLog[orderLog['ars'].isin([1, 7])]
making = orderLog[orderLog['ars'].isin([2, 3])]
else_orders = orderLog[~(orderLog['ars'].isin([1, 2, 3, 7]))]
display('orders with abnormal ars values')
display(else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False))
display(else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False))
# else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False).reset_index()['ars'].values
# kk = else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False)
# kk[kk > 20].index.values
taking['directNum'] = taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].transform('nunique')

if len(taking[taking['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    taking = taking[taking['directNum'] == 1]

assert((taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].nunique() == 1).all() == True)
orderLog = pd.concat([taking, making]).sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    d_el = pd.merge(orderLog, a, on=['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(d_el))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))

### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

display(orderLog[(orderLog['absFilledThisUpdate'] > 0) & (orderLog['updateType'] != 4) & (orderLog['tradePrice'] != -1)].groupby(['date', 'accCode', 'updateType'])['order'].nunique())
orderLog['updateType'] = np.where((orderLog['absFilledThisUpdate'] > 0) & (orderLog['tradePrice'] != -1), 4, orderLog['updateType'])
checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))
display(checkLog[checkLog['status'] == 3].groupby('updateType')['order'].size())
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
try:
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
except:
    print(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
try:
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
except:
    print(orderLog[orderLog['mrss90'] == '-'])
    orderLog = orderLog[orderLog['mrss90'] != '-']
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
try:
    orderLog['aaa'] = orderLog['aaa'].astype(float)
except:
    print(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']   

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['L:\\orderLog\\data\\20201228.pkl' 'L:\\orderLog\\data\\20201231.pkl'
 'L:\\orderLog\\data\\20201229.pkl' 'L:\\orderLog\\data\\20201230.pkl'
 'L:\\orderLog\\data\\20210104.pkl' 'L:\\orderLog\\data\\20210105.pkl'
 'L:\\orderLog\\data\\20210106.pkl' 'L:\\orderLog\\data\\20210107.pkl'
 'L:\\orderLog\\data\\20210108.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
314035,20201228,zs_66_01,6634,2000007,-1,1,49597.0,0,0,-1.0,1.812811e+10
342328,20201228,zs_52_09,5291,2000553,-1,1,53524.0,0,0,-1.0,1.854590e+06
347596,20201228,zs_54_01,5474,2000608,-1,1,47253.0,0,0,-1.0,8.689127e+08
349238,20201228,zs_54_01,5474,2000619,-1,7,47362.0,0,0,-1.0,8.689128e+08
423299,20201228,zs_96_08,9685,2002100,-1,1,48345.0,0,0,-1.0,7.221449e+17
424530,20201228,zs_92_02,9243,2002108,-1,1,40760.0,0,0,-1.0,8.250502e+09
456478,20201228,zs_96_08,9685,2002274,-1,1,53632.0,0,0,-1.0,7.221449e+17
483038,20201228,zs_52_09,5264,2002410,-1,1,34355.0,0,0,-1.0,9.338200e+04
517378,20201228,zs_54_01,5474,2002598,-1,1,53196.0,0,0,-1.0,8.689184e+08
554629,20201228,zs_54_01,5474,2002765,-1,1,53781.0,0,0,-1.0,8.689190e+08


There are orders with all things same except sdd
         Unnamed: 0  ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
1361311    466842.0   1444460.0  0.000687                    0           100   
1361371    466902.0   9795909.0  0.000694                    0           100   

         absOrderSizeCumFilled  accCode  ars             caamd  \
1361311                      0   966301  1.0  1609205575014635   
1361371                      0   966301  1.0  1609208372125359   

         cancellationPending        cfe         clock    clockAtArrival  \
1361311                  0.0  128034.25  1.609206e+15  1609205575014666   
1361371                  0.0   52600.79  1.609208e+15  1609208372125429   

         cumSharesBought  cumSharesBuyInserted  cumSharesSellInserted  \
1361311              0.0                   0.0                  300.0   
1361371              0.0                   0.0                  600.0   

         cumSharesSold      date  finalState        gfe       hee

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

,date,colo,accCode,secid,vai,sdd,clockAtArrival
0,20210108,zs_96_08,6237,1603317,1375708.0,103134000.0,2


accCode
6237    1
Name: date, dtype: int64

99.13% SZE orders triggered by msg data
3. IPO stocks selling (ars = 301, 302)
4. updateType 7 orders
5. updateType 6 orders
6. CYB stocks order size < 30w
7. unexpected updateType


date      accCode  updateType
20201228  8856     2             213
20201229  8856     2              23
20210107  8856     2             357
20210108  8856     2             336
Name: order, dtype: int64

updateType
(0, 2, 1, 1, 3)           14
(0, 2, 1, 3, 4)           22
(0, 2, 1, 4, 1, 3)         1
(0, 2, 1, 4, 3, 4)         4
(0, 2, 1, 6)               2
(0, 2, 4, 1)               6
(0, 2, 4, 1, 1, 3)         2
(0, 2, 4, 1, 3, 4)        20
(0, 2, 4, 1, 4, 3, 4)      5
(0, 2, 4, 3)               6
(0, 2, 6)                 99
(0, 6)                   958
(0, 8)                    32
Name: order, dtype: int64

In [4]:
rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] == 8854) & (rawOrderLog['ars'] == 0)].groupby('date')['secid'].size()

date
20201228    1067
20201229       2
20201230       3
20201231       1
20210108       1
Name: secid, dtype: int64

In [36]:
IF_group = pd.read_table(r'E:\stockGroup_IF.txt', header=None)
IC_group = pd.read_table(r'E:\stockGroup_IC.txt', header=None)
CSI1000_group = pd.read_table(r'E:\stockGroup_CSI1000.txt', header=None)
IF_group1 = IF_group.iloc[0, :].values[0][1:-2].split(', ')
IF_group2 = IF_group.iloc[1, :].values[0][1:-2].split(', ')
IC_group1 = IC_group.iloc[0, :].values[0][1:-2].split(', ')
IC_group2 = IC_group.iloc[1, :].values[0][1:-2].split(', ')
CSI1000_group1 = CSI1000_group.iloc[0, :].values[0][1:-2].split(', ')
CSI1000_group2 = CSI1000_group.iloc[1, :].values[0][1:-2].split(', ')
assert(len(IF_group1) == len(IF_group2))
assert(len(IC_group1) == len(IC_group2))
assert(len(CSI1000_group1) == len(CSI1000_group2))

test1 = orderLog[orderLog['secid'].isin(IF_group1 + IF_group2)]
test1['index_name'] = 'IF'
test1['group_tag'] = np.where(test1['secid'].isin(IF_group1), 1, 2)
display(test1[(test1['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test1[(test1['updateType'] == 0) & (test1['accCode'] == 9461)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test2 = orderLog[orderLog['secid'].isin(IC_group1 + IC_group2)]
test2['index_name'] = 'IC'
test2['group_tag'] = np.where(test2['secid'].isin(IC_group1), 1, 2)
display(test2[(test2['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test2[(test2['updateType'] == 0) & (test2['accCode'] == 9441)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test3 = orderLog[orderLog['secid'].isin(CSI1000_group1 + CSI1000_group2)]
test3['index_name'] = 'CSI1000'
test3['group_tag'] = np.where(test3['secid'].isin(CSI1000_group1), 1, 2)
display(test3[(test3['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test3[(test3['updateType'] == 0) & (test3['accCode'].isin([5456, 9758]))].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())

test = pd.concat([test2[test2['accCode'] == 9441], test3[test3['accCode'] == 5456], 
                  test3[(test3['accCode'] == 9758) & (test3['date'] == 20201221)], test1[(test1['accCode'] == 9461) & (test1['date'] >= 20201221)]])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


colo      accCode  group_tag
zs_52_06  5222     2                 [1.0]
          5269     2                 [1.0]
          5273     2                 [1.0]
          5332     2                 [1.0]
          5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5276     2                 [1.0]
          5281     1                 [1.0]
                   2                 [1.0]
          5292     2                 [1.0]
          5329     2                 [1.0]
          5377     2                 [1.0]
          5386     2                 [1.0]
zs_52_09  5225     2                 [1.0]
          5230     2                 [1.0]
          5264     1                 [1.0]
                   2                 [1.0]
          5290     2                 [1.0]
          5291     2                 [1.0]
          5326     2                 [1.0]
zs_52_13  5329     2                 [1.0]
          5377     2                 [1.0]
          5386     2     

colo      accCode  group_tag
zs_94_03  9461     1            39
                   2            34
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5222     1                      [1.0]
                   2                      [1.0]
          5232     1                      [1.0]
          5269     1                      [1.0]
                   2                      [1.0]
          5273     1                      [1.0]
                   2                      [1.0]
          5275     1                      [1.0]
          5287     1                      [1.0]
          5332     1                      [1.0]
                   2                      [1.0]
          5384     1                      [1.0]
                   2                      [1.0]
zs_52_08  5276     1                      [1.0]
                   2                      [1.0]
          5281     1                      [1.0]
                   2                      [1.0]
          5292     1                      [1.0]
                   2                      [1.0]
          5329     1                      [1.0]
           

colo      accCode  group_tag
zt_94_06  9441     1            38
                   2            34
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5222     1                 [1.0]
                   2                 [1.0]
          5232     1                 [1.0]
                   2                 [1.0]
          5269     1                 [1.0]
                   2                 [1.0]
          5273     1                 [1.0]
                   2                 [1.0]
          5275     1                 [1.0]
                   2                 [1.0]
          5287     1                 [1.0]
                   2                 [1.0]
          5332     1                 [1.0]
                   2                 [1.0]
          5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5276     1                 [1.0]
                   2                 [1.0]
          5281     1                 [1.0]
          5292     1                 [1.0]
                   2                 [1.0]
          5329     1                 [1.0]
                   2     

colo      accCode  group_tag
zs_54_01  5456     1            100
                   2             98
zs_96_06  9758     1            112
                   2            107
Name: secid, dtype: int64

In [64]:
sl = test[(test['updateType'] == 0) & (test['ars'].isin([2,3])) & (test['date'] >= 20210104) & (test['secid'] > 2000000)].groupby(['secid'])['order'].size().reset_index().sort_values(by=['order'], ascending=False)['secid'].values[:10]
check = test[(test['updateType'] == 0) & (test['ars'].isin([2,3])) & (test['date'] >= 20210104) & (test['secid'].isin(sl))]
check.groupby('date')['order'].size()

date
20210104    227
20210105    131
20210106    106
20210107    146
20210108    117
Name: order, dtype: int64

In [65]:
check.groupby('date')['secid'].nunique()

date
20210104    10
20210105    10
20210106     8
20210107     9
20210108    10
Name: secid, dtype: int64

In [61]:
check.groupby(['date', 'secid'])['order'].size().describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).astype(int)

count    50
mean     33
std      14
min       8
10%      18
20%      21
30%      23
40%      27
50%      33
60%      36
70%      38
80%      41
90%      46
max      82
Name: order, dtype: int32

In [13]:
test[(test['updateType'] == 0) & (test['ars'].isin([2,3])) & (test['date'] >= 20210104) & (test['secid'] > 2000000)].groupby(['secid'])['order'].size().reset_index().sort_values(by=['order'], ascending=False)['secid'].values[:10]

array([2002558, 2002032, 2002153, 2300601, 2300487, 2300149, 2300398,
       2002833, 2300347, 2300653], dtype=int64)

In [14]:
sl = test[(test['updateType'] == 0) & (test['ars'].isin([2,3])) & (test['date'] >= 20210104) & (test['secid'] > 2000000)].groupby(['secid'])['order'].size().reset_index().sort_values(by=['order'], ascending=False)['secid'].values[:10]
test1 = test[(test['ars'].isin([2, 3])) & (test['secid'].isin(sl)) & (test['date'] >= 20210104)].sort_values(by = ['date', 'colo', 'accCode', 'secid', 'vai', 'order', 'clockAtArrival'])
test1['sdd'] = test1.groupby(['order'])['sdd'].transform('first')
test1['ApplSeqNum'] = test1.groupby(['order'])['ApplSeqNum'].transform('first')
test1['datetime'] = (test1['date'] * 1000000000 + test1['sdd'].astype('int64')).astype('int64').astype(str)
test1['datetime'] = pd.to_datetime(test1['datetime'], format='%Y%m%d%H%M%S%f')
test1["start_time"] = test1['datetime'].apply(lambda x: np.int64((x + datetime.timedelta(hours=-8)).timestamp() * 1e6))
test1['caamd'] = test1.groupby('order')['caamd'].transform('first')
test1['mdClockAtArrival'] = test1['clockAtArrival'] - test1['caamd'] + test1['start_time']
test1['time'] = test1['mdClockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
# test1 = test1[test1['updateType'].isin([0, 4, 3])]
test1 = test1.sort_values(by=['date', 'secid', 'vai', 'colo', 'accCode', 'order', 'clockAtArrival']).reset_index(drop=True)
test1['innerSeq'] = test1.index
test1[['date', 'vai', 'secid', 'accCode', 'updateType', 'order', 'sdd', 'ApplSeqNum', 'orderPrice', 'tradePrice', 'absOrderSize', 'absFilledThisUpdate', 'duration']].reset_index(drop=True)

,date,vai,secid,accCode,updateType,order,sdd,ApplSeqNum,orderPrice,tradePrice,absOrderSize,absFilledThisUpdate,duration
0,20210104,86486.0,2002032,9461,0,903869,93126980.0,1206512.0,77.85,-1.0,300,0,0
1,20210104,86486.0,2002032,9461,2,903869,93126980.0,1206512.0,77.85,-1.0,300,0,4258
2,20210104,86486.0,2002032,9461,1,903869,93126980.0,1206512.0,77.85,-1.0,300,0,1664852
3,20210104,86486.0,2002032,9461,3,903869,93126980.0,1206512.0,77.85,-1.0,300,0,1667679
4,20210104,87886.0,2002032,9461,0,903870,93128910.0,1220143.0,77.92,-1.0,300,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,20210108,1205762.0,2300653,5456,1,1730785,142508330.0,24714189.0,53.75,-1.0,100,0,7821679
2876,20210108,1205762.0,2300653,5456,3,1730785,142508330.0,24714189.0,53.75,-1.0,100,0,7824198
2877,20210108,1205962.0,2300653,5456,0,1730786,142517940.0,24726287.0,53.80,-1.0,100,0,0
2878,20210108,1205962.0,2300653,5456,2,1730786,142517940.0,24726287.0,53.80,-1.0,100,0,2637


In [15]:
for d in test1['date'].unique():
    test1[test1['date'] == d][['date', 'vai', 'secid', 'updateType', 'order', 'sdd', 'ApplSeqNum', 'orderPrice', 'tradePrice', 'absOrderSize', 'absFilledThisUpdate', 'duration']].reset_index(drop=True).to_csv('A:\\temp\\makingOrder\\' + str(d) + '.csv', index=False)

In [2]:
orderLog[(orderLog['mrstaat'].isin([11000, 13000])) & (orderLog['secid'] > 2000000)].groupby('date')['colo_broker'].unique()

date
20201228    [zs_53, zs_97, zs_52, zs_66, zs_89, zs_62]
20201229    [zs_97, zs_53, zs_52, zs_66, zs_89, zs_62]
20201230    [zs_97, zs_53, zs_52, zs_89, zs_66, zs_62]
20201231    [zs_97, zs_52, zs_53, zs_89, zs_66, zs_62]
20210104    [zs_52, zs_97, zs_53, zs_89, zs_66, zs_62]
20210105    [zs_52, zs_97, zs_89, zs_53, zs_66, zs_62]
20210106    [zs_52, zs_97, zs_53, zs_89, zs_66, zs_62]
20210107    [zs_97, zs_53, zs_52, zs_66, zs_89, zs_62]
20210108    [zs_52, zs_97, zs_53, zs_89, zs_66, zs_62]
Name: colo_broker, dtype: object

In [18]:
# fill rate by exchange and daily Turnover
a = orderLog[(orderLog['updateType'] == 0)]['orderNtl'].sum()
b = orderLog[(orderLog['updateType'] == 4)]['tradeNtl'].sum()
print('total %.0f%%'%(b/a*100))

a = orderLog[(orderLog['exchange'] == 'SSE') & (orderLog['updateType'] == 0)]['orderNtl'].sum()
b = orderLog[(orderLog['exchange'] == 'SSE') & (orderLog['updateType'] == 4)]['tradeNtl'].sum()
print('SSE %.0f%%'%(b/a*100))

a = orderLog[(orderLog['exchange'] == 'SZE') & (orderLog['updateType'] == 0)]['orderNtl'].sum()
b = orderLog[(orderLog['exchange'] == 'SZE') & (orderLog['updateType'] == 4)]['tradeNtl'].sum()
print('SZE %.0f%%'%(b/a*100))

a = orderLog[(orderLog['exchange'] == 'SSE') & (orderLog['updateType'] == 0) & (orderLog['colo_broker'] == 'zt_52')]['orderNtl'].sum()
b = orderLog[(orderLog['exchange'] == 'SSE') & (orderLog['updateType'] == 4) & (orderLog['colo_broker'] == 'zt_52')]['tradeNtl'].sum()
print('zt_52 %.0f%%'%(b/a*100))

a = orderLog[(orderLog['exchange'] == 'SZE') & (orderLog['updateType'] == 0) & (orderLog['colo_broker'] == 'zs_52')]['orderNtl'].sum()
b = orderLog[(orderLog['exchange'] == 'SZE') & (orderLog['updateType'] == 4) & (orderLog['colo_broker'] == 'zs_52')]['tradeNtl'].sum()
print('zs_52 %.0f%%'%(b/a*100))

a = orderLog.groupby(['date', 'exchange', 'colo_broker'])['tradeNtl'].sum().reset_index()
l = len(orderLog["date"].unique())
exchangeTurnover = a.groupby(['date', 'exchange'])['tradeNtl'].sum().reset_index()
exchangeTurnover = exchangeTurnover.groupby(['exchange'])['tradeNtl'].mean().reset_index()
exchangeTurnover['tradeNtl'] = (exchangeTurnover['tradeNtl']/10000).astype('int64')
exchangeTurnover.columns = ['exchange', 'exchange turnover']
exchangeTurnover = pd.merge(a, exchangeTurnover, how='left', on=['exchange'], validate='many_to_one')
pp = exchangeTurnover.groupby(['exchange', 'exchange turnover', 'colo_broker'])["tradeNtl"].count().reset_index()
exchangeTurnover = (exchangeTurnover.groupby(['exchange', 'exchange turnover', 'colo_broker'])['tradeNtl'].mean()/10000).astype('int64').reset_index()
exchangeTurnover.columns = ['exchange', 'exchange turnover', 'colo_broker', 'turnover']
exchangeTurnover = pd.merge(exchangeTurnover, pp, on=['exchange', 'exchange turnover', 'colo_broker'])
exchangeTurnover['turnover %'] = (exchangeTurnover['turnover'] *  exchangeTurnover['tradeNtl'])/ (exchangeTurnover['exchange turnover']*l)
exchangeTurnover = exchangeTurnover.rename(columns={'colo_broker': 'broker'})
exchangeTurnover = exchangeTurnover.groupby(['exchange', 'exchange turnover', 'broker'])[['turnover', 'turnover %']].first()
exchangeTurnover['turnover %'] = exchangeTurnover['turnover %'].apply(lambda x: '%.1f%%'%(100*x))

from IPython.display import display, HTML
display(HTML(exchangeTurnover.to_html()))

total 76%
SSE 73%
SZE 78%
zt_52 73%
zs_52 82%


In [2]:
# order speed comparison between brokers (under same sta and same mrstauc)
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['duration'] <= 1e6]
# checkLog = checkLog[checkLog['colo_broker'].isin(['zs_88', 'zs_54'])]

sta_list = orderLog['sta_uc'].unique()

for exchg in ['SH','SZ']: 
    print(exchg)
    result = {}
    for col in ['date', 'broker1', 'broker2', 'isFast', 'sta']:
        result[col] = []
    for i in sta_list:
        checkLog1 = checkLog[checkLog['sta_uc'] == i]
        checkLog1['brokerNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['colo_broker'].transform('nunique')
        checkLog1 = checkLog1[checkLog1['brokerNum'] >= 2]
        checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
        checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                        np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
        checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'accCode', 'secid', 'vai', 'colo_broker', 'status']].first().reset_index()

        if exchg == 'SH':
            exchangeLog1 = checkLog1[(checkLog1['secid'] < 2000000) & (checkLog1['colo_broker'].str[:2] == 'zt')]
        else:
            exchangeLog1 = checkLog1[(checkLog1['secid'] >= 2000000) & (checkLog1['colo_broker'].str[:2] == 'zs')]

        for group, groupData in exchangeLog1.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['colo_broker'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]
                        result['sta'] += [i, i]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]   
                            

    result = pd.DataFrame(result)
    result = result[result['isFast'] != -1]
    result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    result['faster'] = result['fasterSum']/result['count']

    summary = pd.DataFrame()
    summaryCols = []
    countCols = []
    fasterCols = []
    for date in result['date'].unique():
        a = result[result['date'] == date].reset_index(drop=True)
        a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
        a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
        a['faster'] = a['fasterSum']/a['count']
        a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
        a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
        summaryCols += ['%s count'%date, '%s faster'%date]
        countCols += ['%s count'%date]
        fasterCols += ['%s faster'%date]

        if summary.empty:
            summary = a.copy()
        else:
            summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

    a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', 'total count', 'total faster']
    summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
    summaryCols = ['total count', 'total faster'] + summaryCols

    savePath = r'L:\orderLog\result\relativeSpeed'
    a = summary.groupby(['broker1', 'broker2'])[['total count', 'total faster']].first().reset_index()
    a.to_csv(os.path.join(savePath, 'relativeSpeedBroker_%s_%s_%s.csv'%(startDate, endDate, exchg)), index=False)

    countCols += ['total count']
    fasterCols += ['total faster']
    summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first().sort_values(by=['broker1', 'broker2'])
    for col in countCols:
        summary[col] = summary[col].fillna(0)
        summary[col] = summary[col].astype('int64')

    for col in fasterCols:
        summary[col] = summary[col].fillna(-1)
        summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
        summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])
    from IPython.display import display, HTML
    display(HTML(summary.iloc[:, :2].to_html()))

SH


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

SZ


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [39]:
result.groupby(['date', 'sta'])['count'].size()

date      sta           
20201228  1. staone_0        68
          2. statwo_1000     42
20201229  1. staone_0        10
          2. statwo_1000      2
20201230  1. staone_0        16
          2. statwo_1000     24
20201231  1. staone_0        16
          2. statwo_1000     52
20210104  1. staone_0         8
          2. statwo_1000      6
20210105  1. staone_0        16
          2. statwo_1000      2
20210106  1. staone_0        24
          2. statwo_1000      4
20210107  1. staone_0       100
          2. statwo_1000     82
20210108  1. staone_0        98
          2. statwo_1000    126
Name: count, dtype: int64

In [18]:
orderLog[(orderLog['colo_broker'] == 'zs_54') & (orderLog['exchange'] == 'SZE') & (orderLog['updateType'] == 0)].groupby(['date', 'sta'])['order'].size()

date      sta      
20201214  1. staone    5630
20201215  1. staone    4827
20201216  1. staone    5869
20201217  1. staone    6002
20201218  1. staone    5501
20201221  1. staone    5723
20201222  1. staone    6241
20201223  1. staone    6697
20201224  1. staone    6910
20201225  1. staone    6741
Name: order, dtype: int64

In [52]:
# order speed comparison between brokers (under same sta and same mrstauc)
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['duration'] <= 1e6]
checkLog = checkLog[checkLog['server_account'].isin(['zs8804_8856', 'zs7201_7293'])]

sta_list = orderLog['sta_uc'].unique()

for exchg in ['SZ']: 
    print(exchg)
    result = {}
    for col in ['date', 'broker1', 'broker2', 'isFast', 'sta']:
        result[col] = []
    for i in sta_list:
        checkLog1 = checkLog[checkLog['sta_uc'] == i]
        checkLog1['brokerNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
        checkLog1 = checkLog1[checkLog1['brokerNum'] >= 2]
        checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
        checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                        np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
        checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'accCode', 'secid', 'vai', 'server_account', 'status']].first().reset_index()

        if exchg == 'SH':
            exchangeLog1 = checkLog1[checkLog1['secid'] < 2000000]
        else:
            exchangeLog1 = checkLog1[checkLog1['secid'] >= 2000000]

        for group, groupData in exchangeLog1.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['server_account'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]
                        result['sta'] += [i, i]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]   

    result = pd.DataFrame(result)
    result = result[result['isFast'] != -1]
    result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    result['faster'] = result['fasterSum']/result['count']

    summary = pd.DataFrame()
    summaryCols = []
    countCols = []
    fasterCols = []
    for date in result['date'].unique():
        a = result[result['date'] == date].reset_index(drop=True)
        a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
        a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
        a['faster'] = a['fasterSum']/a['count']
        a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
        a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
        summaryCols += ['%s count'%date, '%s faster'%date]
        countCols += ['%s count'%date]
        fasterCols += ['%s faster'%date]

        if summary.empty:
            summary = a.copy()
        else:
            summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

    a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', 'total count', 'total faster']
    summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
    summaryCols = ['total count', 'total faster'] + summaryCols

    savePath = r'L:\orderLog\result\relativeSpeed'
    a = summary.groupby(['broker1', 'broker2'])[['total count', 'total faster']].first().reset_index()
    a.to_csv(os.path.join(savePath, 'relativeSpeedBroker1_%s_%s_%s.csv'%(startDate, endDate, exchg)), index=False)

    countCols += ['total count']
    fasterCols += ['total faster']
    summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first().sort_values(by=['broker1', 'broker2'])
    for col in countCols:
        summary[col] = summary[col].fillna(0)
        summary[col] = summary[col].astype('int64')

    for col in fasterCols:
        summary[col] = summary[col].fillna(-1)
        summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
        summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])
    from IPython.display import display, HTML
    display(HTML(summary.iloc[:, :2].to_html()))

SZ


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,,total count,total faster
broker1,broker2,,
zs7201_7293,zs8804_8856,170,74%
zs8804_8856,zs7201_7293,170,26%


In [64]:
checkLog1['num'] = checkLog1.groupby('group')['status'].transform('nunique')
checkLog1[(checkLog1['num'] > 1)].head()

,group,order,date,accCode,secid,vai,server_account,status,num
4,49087,103173,20201228,7293,2000543,67042134.0,zs7201_7293,1,2
5,49087,103477,20201228,8856,2000543,67042134.0,zs8804_8856,2,2
10,51777,103199,20201228,7293,2000681,4442554.0,zs7201_7293,0,2
11,51777,103495,20201228,8856,2000681,4442554.0,zs8804_8856,2,2
12,51778,103200,20201228,7293,2000681,4473954.0,zs7201_7293,0,2


In [14]:
check = orderLog[orderLog['accCode'] == 8856].groupby(['date', 'order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
check[check['date'].isin([20201228, 20201229, 20210107, 20210108])].groupby(['date', 'updateType'])['order'].size()

date      updateType     
20201228  (0, 2, 1, 3)        57
          (0, 2, 2)          201
          (0, 2, 2, 1, 3)      3
20201229  (0, 2, 1)            1
          (0, 2, 1, 3)        14
          (0, 2, 2)           23
20210107  (0, 2, 1, 3)        96
          (0, 2, 2)          333
          (0, 2, 2, 1, 3)      2
20210108  (0,)                 1
          (0, 2, 1, 3)        49
          (0, 2, 2)          313
          (0, 2, 2, 1, 3)      3
Name: order, dtype: int64

In [3]:
rawOrderLog[(rawOrderLog['internalId'] == 47) & (rawOrderLog['accCode'] == 8856) & (rawOrderLog['date'] == 20210107)][['internalId', 'orderId', 'date', 'updateType', 'accCode', 'vai', 'secid', 'absOrderSize', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'clockAtArrival']].sort_values(by='clockAtArrival')

,internalId,orderId,date,updateType,accCode,vai,secid,absOrderSize,absFilledThisUpdate,orderPrice,tradePrice,clockAtArrival


In [6]:
orderLog[(orderLog['accCode'] == 8856) & (orderLog['date'] >= 20210107)][['internalId', 'orderId', 'date', 'updateType', 'accCode', 'vai', 'secid', 'absOrderSize', 'absFilledThisUpdate', 'duration', 'orderPrice', 'tradePrice']].head(100)

,internalId,orderId,date,updateType,accCode,vai,secid,absOrderSize,absFilledThisUpdate,duration,orderPrice,tradePrice
5949515,58.0,58.0,20210107,0,8856,158700.0,2000028,100,0,0,44.34,-1.00
5949516,58.0,58.0,20210107,2,8856,158700.0,2000028,100,0,3788,44.34,-1.00
5949517,58.0,58.0,20210107,1,8856,158700.0,2000028,100,0,1024958,44.34,-1.00
5949518,58.0,58.0,20210107,3,8856,158700.0,2000028,100,0,1028618,44.34,-1.00
5950020,369.0,369.0,20210107,0,8856,3304497.0,2000028,100,0,0,43.55,-1.00
5950021,369.0,369.0,20210107,2,8856,3304497.0,2000028,100,0,2985,43.55,-1.00
5950022,369.0,369.0,20210107,1,8856,3304497.0,2000028,100,0,1077155,43.55,-1.00
5950023,369.0,369.0,20210107,3,8856,3304497.0,2000028,100,0,1080768,43.55,-1.00
5951393,47.0,47.0,20210107,0,8856,1299500.0,2000039,100,0,0,15.95,-1.00
5951394,47.0,47.0,20210107,2,8856,1299500.0,2000039,100,0,3403,15.95,-1.00


In [70]:
check = orderLog[orderLog['order'].isin(checkLog1[(checkLog1['num'] > 1) & (checkLog1['accCode'] == 8856)]['order'].unique())]
# check = check.groupby(['date', 'order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
# check.groupby(['date', 'updateType'])['order'].size()

In [73]:
check[['date', 'updateType', 'accCode', 'vai', 'secid', 'absOrderSize', 'absFilledThisUpdate', 'duration', 'orderPrice', 'tradePrice']].head(100)

,date,updateType,accCode,vai,secid,absOrderSize,absFilledThisUpdate,duration,orderPrice,tradePrice
337754,20201228,0,8856,67042134.0,2000543,200,0,0,4.40,-1.00
337755,20201228,2,8856,67042134.0,2000543,200,0,3824,4.40,-1.00
337756,20201228,2,8856,67042134.0,2000543,200,200,3918,4.40,4.40
355127,20201228,0,8856,4442554.0,2000681,400,0,0,12.28,-1.00
355128,20201228,2,8856,4442554.0,2000681,400,0,3651,12.28,-1.00
355129,20201228,2,8856,4442554.0,2000681,400,400,3769,12.28,12.28
355133,20201228,0,8856,4473954.0,2000681,100,0,0,12.29,-1.00
355134,20201228,2,8856,4473954.0,2000681,100,0,2650,12.29,-1.00
355135,20201228,2,8856,4473954.0,2000681,100,100,2725,12.29,12.29
355147,20201228,0,8856,4536754.0,2000681,100,0,0,12.30,-1.00


In [ ]:
orderLog[(orderLog['date'].isin([20201228, 2020122]))]

In [65]:
orderLog[orderLog['order'] == 103495][['updateType', 'accCode', 'vai', 'secid', 'absOrderSize', 'absFilledThisUpdate', 'duration', 'orderPrice', 'tradePrice']]

,updateType,accCode,vai,secid,absOrderSize,absFilledThisUpdate,duration,orderPrice,tradePrice
355127,0,8856,4442554.0,2000681,400,0,0,12.28,-1.00
355128,2,8856,4442554.0,2000681,400,0,3651,12.28,-1.00
355129,2,8856,4442554.0,2000681,400,400,3769,12.28,12.28


In [37]:
summary = summary.reset_index()
summary[(summary['broker1'].str[:4] == 'zs88') & (summary['broker2'].str[:4] == 'zs52')].sort_values(by='total count', ascending=False)

,broker1,broker2,total count,total faster,20201228 count,20201228 faster,20201229 count,20201229 faster,20201230 count,20201230 faster,20201231 count,20201231 faster,20210104 count,20210104 faster,20210105 count,20210105 faster,20210106 count,20210106 faster,20210107 count,20210107 faster,20210108 count,20210108 faster
1,zs8804_8856,zs5206_5332,132,30%,37,0%,10,0%,28,89%,2,100%,5,100%,5,80%,4,100%,8,0%,33,0%


In [45]:
# summary = summary.reset_index()
summary[(summary['broker1'] == 'zs8804_8865') & (summary['broker2'].isin(['zs5206_5232', 'zs5206_5275', 'zs5401_5456', 'zs9606_9758']))].sort_values(by='total count', ascending=False)

,level_0,index,broker1,broker2,total count,total faster,20201228 count,20201228 faster,20201229 count,20201229 faster,20201230 count,20201230 faster,20201231 count,20201231 faster,20210104 count,20210104 faster,20210105 count,20210105 faster,20210106 count,20210106 faster,20210107 count,20210107 faster,20210108 count,20210108 faster
555,555,555,zs8804_8865,zs5206_5232,3126,64%,391,0%,0,,352,55%,150,70%,385,64%,417,82%,481,83%,488,72%,462,81%
556,556,556,zs8804_8865,zs5206_5275,1178,51%,162,2%,0,,87,63%,101,75%,126,46%,68,76%,111,60%,246,42%,277,65%
562,562,562,zs8804_8865,zs9606_9758,446,72%,53,0%,0,,43,86%,33,91%,30,80%,15,93%,35,83%,81,77%,156,81%
558,558,558,zs8804_8865,zs5401_5456,279,71%,41,20%,0,,40,60%,0,,36,72%,56,98%,52,79%,32,72%,22,91%


In [8]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'] // 100 == 53)].groupby(['colo', 'accCode', 'sta'])['date'].size()

colo      accCode  sta      
zs_52_06  5332     1. staone     2471
                   2. statwo     6571
          5384     1. staone      668
                   2. statwo    16191
zs_52_08  5377     1. staone      890
                   2. statwo    14748
          5386     1. staone      520
                   2. statwo    11485
Name: date, dtype: int64

In [42]:
kk = summary.reset_index()
kk['b1'] = kk['broker1'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
kk['b2'] = kk['broker2'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
display(kk[(kk['b1'] == 'zs88') & (kk['b2'].isin(['zs52']))].sort_values(by='total count', ascending=False))

,broker1,broker2,total count,total faster,20201130 count,20201130 faster,20201201 count,20201201 faster,20201202 count,20201202 faster,20201203 count,20201203 faster,20201204 count,20201204 faster,20201207 count,20201207 faster,20201208 count,20201208 faster,20201209 count,20201209 faster,20201210 count,20201210 faster,20201211 count,20201211 faster,b1,b2
441,zs8804_8865,zs5206_5287,1507,0%,422,1%,281,1%,273,0%,323,0%,25,0%,37,0%,22,0%,22,0%,42,2%,60,0%,zs88,zs52
439,zs8804_8865,zs5206_5232,1260,0%,13,0%,5,0%,3,0%,4,0%,116,1%,239,1%,227,0%,173,0%,230,1%,250,0%,zs88,zs52
440,zs8804_8865,zs5206_5275,715,1%,118,0%,111,0%,61,0%,63,0%,76,3%,67,1%,19,0%,46,0%,76,0%,78,3%,zs88,zs52
426,zs8804_8856,zs5208_5276,324,35%,76,37%,34,41%,30,20%,14,43%,10,20%,16,56%,14,36%,45,33%,40,32%,45,38%,zs88,zs52
430,zs8804_8856,zs5209_5289,179,17%,39,18%,17,6%,7,29%,9,11%,0,,20,15%,8,38%,23,26%,26,15%,30,13%,zs88,zs52
425,zs8804_8856,zs5208_5229,170,41%,44,30%,22,27%,1,100%,12,75%,3,33%,12,50%,12,50%,31,45%,17,53%,16,31%,zs88,zs52
431,zs8804_8856,zs5209_5291,170,34%,39,44%,13,23%,10,10%,8,25%,6,0%,10,40%,8,50%,18,44%,23,30%,35,31%,zs88,zs52
427,zs8804_8856,zs5208_5292,147,66%,30,67%,14,71%,17,76%,9,78%,2,50%,6,83%,10,50%,28,57%,16,75%,15,53%,zs88,zs52


In [20]:
orderLog[(orderLog['colo'] == 'zs_88_04') & (orderLog['updateType'] == 0)].groupby(['accCode'])['date'].size()

accCode
8856       1961
8865      20496
8924      28616
8967      25172
892402    15935
896702    16556
Name: date, dtype: int64

In [112]:
orderLog[(orderLog['accCode'] // 100 == 89) & (orderLog['exchange'] == 'SSE') & (orderLog['updateType'] == 0)].groupby('colo')['accCode'].unique()

colo
zs_88_04          [8967, 8924]
zt_88_06    [8970, 8971, 8943]
Name: accCode, dtype: object

In [40]:
orderLog[(orderLog['colo'].str[:5] == 'zt_94') & (orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SSE')].groupby(['date', 'sta', 'colo'])['date'].size()

date      sta        colo    
20201130  1. staone  zt_94_06     228
          2. statwo  zt_94_06    1395
          3. sta300  zt_94_06    2572
20201201  1. staone  zt_94_06     116
          2. statwo  zt_94_06    1023
          3. sta300  zt_94_06    1899
20201202  1. staone  zt_94_06     158
          2. statwo  zt_94_06    1226
          3. sta300  zt_94_06    1677
20201203  1. staone  zt_94_06     122
          2. statwo  zt_94_06     719
          3. sta300  zt_94_06    1621
20201204  1. staone  zt_94_06      98
          2. statwo  zt_94_06     667
          3. sta300  zt_94_06    1236
20201207  1. staone  zt_94_06     186
          2. statwo  zt_94_06    1114
          3. sta300  zt_94_06    1317
20201208  1. staone  zt_94_06      92
          2. statwo  zt_94_06     672
          3. sta300  zt_94_06     891
20201209  1. staone  zt_94_06     120
          2. statwo  zt_94_06    1405
          3. sta300  zt_94_06    1706
20201210  1. staone  zt_94_06     216
          2. statwo 

In [37]:
orderLog[orderLog['colo'].str[:5] == 'zt_88']['colo'].unique()

array(['zt_88_06', 'zt_88_03'], dtype=object)

In [87]:
kk = summary.reset_index()
kk['b1'] = kk['broker1'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
kk['b2'] = kk['broker2'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
display(kk[(kk['b1'] == 'zt94') & (kk['b2'] == 'zt52')].sort_values(by='total count', ascending=False))

,broker1,broker2,total count,total faster,20201130 count,20201130 faster,20201201 count,20201201 faster,20201202 count,20201202 faster,20201203 count,20201203 faster,20201204 count,20201204 faster,20201207 count,20201207 faster,20201208 count,20201208 faster,20201209 count,20201209 faster,20201210 count,20201210 faster,20201211 count,20201211 faster,b1,b2
520,zt9406_9441,zt5210_522401,685,48%,72,32%,68,59%,87,59%,49,33%,31,39%,27,48%,39,62%,130,42%,125,54%,57,44%,zt94,zt52
517,zt9406_9441,zt5204_522601,678,47%,67,31%,66,55%,67,60%,51,43%,32,47%,20,50%,53,55%,126,50%,137,47%,59,36%,zt94,zt52
519,zt9406_9441,zt5207_523001,325,50%,39,36%,39,56%,42,64%,26,42%,16,56%,18,56%,22,59%,55,47%,42,43%,26,42%,zt94,zt52
521,zt9406_9441,zt5210_528901,309,51%,25,36%,27,63%,42,48%,26,38%,17,59%,28,68%,15,60%,44,52%,49,49%,36,44%,zt94,zt52
518,zt9406_9441,zt5204_527301,301,54%,23,48%,36,58%,38,63%,21,33%,10,70%,21,52%,17,65%,61,51%,49,55%,25,52%,zt94,zt52
516,zt9406_9441,zt5204_522201,176,56%,6,33%,15,60%,31,71%,10,50%,4,25%,24,62%,16,56%,26,62%,26,46%,18,44%,zt94,zt52


In [9]:
orderLog[(orderLog['server_account'].isin(['zs9608_9754', 'zt8806_8886'])) & (orderLog['updateType'] == 0)].groupby(['server_account', 'date', 'sta'])['orderPrice'].size()

server_account  date      sta      
zs9608_9754     20201130  1. staone     848
                          2. statwo    5751
                20201201  1. staone     493
                          2. statwo    4173
                20201202  1. staone     346
                          2. statwo    3453
                20201203  1. staone     440
                          2. statwo    3179
                20201204  1. staone     207
                          2. statwo    2639
                20201207  1. staone     108
                          2. statwo    1716
                20201208  1. staone      31
                          2. statwo    1780
                20201209  1. staone     166
                          2. statwo    4044
                20201210  1. staone     138
                          2. statwo    2869
                20201211  1. staone      72
                          2. statwo    2792
zt8806_8886     20201130  1. staone     223
                          2. statwo    1

In [35]:
kk = summary.reset_index()
kk['b1'] = kk['broker1'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
kk['b2'] = kk['broker2'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
display(kk[(kk['b1'] == 'zs52') & (kk['b2'] == 'zs54')].sort_values(by='total count', ascending=False))
display(kk[(kk['b1'] == 'zs53') & (kk['b2'] == 'zs89')].sort_values(by='total count', ascending=False))
display(kk[(kk['b1'] == 'zs88') & (kk['b2'] == 'zs97')].sort_values(by='total count', ascending=False))

,broker1,broker2,total count,total faster,20201116 count,20201116 faster,20201117 count,20201117 faster,20201118 count,20201118 faster,20201119 count,20201119 faster,20201120 count,20201120 faster,20201123 count,20201123 faster,20201124 count,20201124 faster,20201125 count,20201125 faster,20201126 count,20201126 faster,20201127 count,20201127 faster,b1,b2
75,zs5206_5275,zs5401_5456,190,92%,23,87%,24,83%,28,93%,23,91%,39,100%,39,92%,1,0%,5,100%,4,100%,4,75%,zs52,zs54
50,zs5206_5269,zs5401_5474,145,64%,8,75%,2,50%,9,100%,15,67%,23,83%,25,84%,27,44%,12,67%,9,0%,15,47%,zs52,zs54
270,zs5209_5230,zs5401_5474,108,81%,7,86%,10,100%,12,100%,8,88%,11,73%,11,91%,18,61%,11,91%,7,57%,13,77%,zs52,zs54
304,zs5209_5290,zs5401_5474,78,72%,9,89%,7,100%,0,,12,83%,9,56%,5,80%,12,67%,8,88%,6,17%,10,60%,zs52,zs54
255,zs5209_5226,zs5401_5474,62,79%,9,89%,5,100%,6,100%,13,85%,5,80%,6,100%,6,33%,2,100%,5,20%,5,80%,zs52,zs54
49,zs5206_5269,zs5401_5470,49,82%,1,100%,1,0%,7,86%,8,75%,3,100%,14,93%,11,64%,4,100%,0,,0,,zs52,zs54
269,zs5209_5230,zs5401_5470,49,94%,4,100%,4,100%,12,100%,6,83%,5,100%,6,100%,6,83%,4,100%,1,100%,1,0%,zs52,zs54
65,zs5206_5273,zs5401_5474,47,60%,3,67%,4,75%,3,100%,5,80%,0,,1,100%,8,38%,8,50%,4,50%,11,55%,zs52,zs54
240,zs5209_5225,zs5401_5474,46,76%,5,100%,5,100%,4,100%,4,75%,4,50%,5,100%,5,60%,3,100%,5,40%,6,50%,zs52,zs54
7,zs5206_5222,zs5401_5474,40,43%,3,67%,3,67%,1,100%,5,40%,3,33%,4,100%,5,20%,5,60%,5,0%,6,17%,zs52,zs54


,broker1,broker2,total count,total faster,20201116 count,20201116 faster,20201117 count,20201117 faster,20201118 count,20201118 faster,20201119 count,20201119 faster,20201120 count,20201120 faster,20201123 count,20201123 faster,20201124 count,20201124 faster,20201125 count,20201125 faster,20201126 count,20201126 faster,20201127 count,20201127 faster,b1,b2
226,zs5208_5386,zs8804_8924,114,18%,31,16%,26,12%,18,28%,23,22%,3,33%,13,15%,0,,0,,0,,0,,zs53,zs89
126,zs5206_5332,zs8804_8924,84,99%,0,,18,100%,17,100%,8,88%,4,100%,6,100%,7,100%,3,100%,0,,21,100%,zs53,zs89
227,zs5208_5386,zs8804_896702,70,29%,16,19%,11,45%,6,50%,9,44%,18,17%,10,20%,0,,0,,0,,0,,zs53,zs89
127,zs5206_5332,zs8804_896702,63,100%,0,,8,100%,7,100%,3,100%,3,100%,7,100%,8,100%,6,100%,6,100%,15,100%,zs53,zs89
110,zs5206_5328,zs8804_8924,6,100%,0,,4,100%,2,100%,0,,0,,0,,0,,0,,0,,0,,zs53,zs89
111,zs5206_5328,zs8804_896702,6,100%,0,,4,100%,2,100%,0,,0,,0,,0,,0,,0,,0,,zs53,zs89
209,zs5208_5377,zs8804_896702,1,0%,0,,0,,0,,0,,0,,0,,0,,0,,0,,1,0%,zs53,zs89


,broker1,broker2,total count,total faster,20201116 count,20201116 faster,20201117 count,20201117 faster,20201118 count,20201118 faster,20201119 count,20201119 faster,20201120 count,20201120 faster,20201123 count,20201123 faster,20201124 count,20201124 faster,20201125 count,20201125 faster,20201126 count,20201126 faster,20201127 count,20201127 faster,b1,b2
476,zs8804_8856,zs9608_9754,297,98%,45,98%,51,100%,51,100%,18,94%,7,100%,11,100%,22,95%,25,92%,23,100%,44,98%,zs88,zs97


In [12]:
kk = summary.reset_index()
kk['b1'] = kk['broker1'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
kk['b2'] = kk['broker2'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
display(kk[(kk['b1'] == 'zs52') & (kk['b2'] == 'zs54')].sort_values(by='total count', ascending=False))
display(kk[(kk['b1'] == 'zs53') & (kk['b2'] == 'zs89')].sort_values(by='total count', ascending=False))
display(kk[(kk['b1'] == 'zs88') & (kk['b2'] == 'zs97')].sort_values(by='total count', ascending=False))

,broker1,broker2,total count,total faster,20201130 count,20201130 faster,20201201 count,20201201 faster,20201202 count,20201202 faster,20201203 count,20201203 faster,20201204 count,20201204 faster,20201207 count,20201207 faster,20201208 count,20201208 faster,20201209 count,20201209 faster,20201210 count,20201210 faster,20201211 count,20201211 faster,b1,b2
34,zs5206_5269,zs5401_5474,160,44%,25,40%,15,13%,12,25%,14,43%,17,47%,14,50%,13,46%,7,29%,20,40%,23,78%,zs52,zs54
274,zs5209_5290,zs5401_5474,94,62%,18,50%,12,33%,6,67%,7,100%,6,67%,5,40%,7,71%,5,80%,7,43%,21,76%,zs52,zs54
234,zs5209_5230,zs5401_5474,87,68%,17,65%,10,20%,7,71%,8,88%,5,80%,4,75%,8,88%,4,100%,9,44%,15,80%,zs52,zs54
17,zs5206_5232,zs5401_5456,81,49%,0,,2,50%,5,20%,8,25%,22,59%,12,83%,18,39%,1,100%,10,50%,3,0%,zs52,zs54
219,zs5209_5226,zs5401_5474,76,63%,13,46%,7,43%,3,67%,3,67%,7,86%,10,80%,8,75%,3,33%,9,56%,13,69%,zs52,zs54
7,zs5206_5222,zs5401_5474,60,40%,15,33%,8,12%,5,20%,0,,6,83%,2,0%,5,60%,4,0%,7,57%,8,62%,zs52,zs54
203,zs5209_5225,zs5401_5470,56,71%,2,50%,2,50%,7,29%,9,67%,12,83%,10,100%,3,67%,0,,7,57%,4,100%,zs52,zs54
204,zs5209_5225,zs5401_5474,45,47%,9,22%,6,33%,1,100%,0,,6,67%,1,0%,6,50%,3,67%,9,56%,4,50%,zs52,zs54
49,zs5206_5273,zs5401_5474,42,24%,12,25%,7,0%,0,,0,,4,50%,1,100%,3,0%,4,25%,5,20%,6,33%,zs52,zs54
48,zs5206_5273,zs5401_5470,41,44%,1,0%,1,0%,8,12%,7,29%,9,67%,5,60%,1,100%,0,,8,50%,1,100%,zs52,zs54


,broker1,broker2,total count,total faster,20201130 count,20201130 faster,20201201 count,20201201 faster,20201202 count,20201202 faster,20201203 count,20201203 faster,20201204 count,20201204 faster,20201207 count,20201207 faster,20201208 count,20201208 faster,20201209 count,20201209 faster,20201210 count,20201210 faster,20201211 count,20201211 faster,b1,b2
100,zs5206_5384,zs8804_892402,321,100%,34,100%,26,100%,43,100%,29,100%,17,100%,34,100%,36,100%,49,100%,15,100%,38,100%,zs53,zs89
91,zs5206_5332,zs8804_8924,90,100%,27,100%,12,100%,16,100%,6,100%,3,100%,3,100%,3,100%,4,100%,11,100%,5,100%,zs53,zs89
92,zs5206_5332,zs8804_896702,26,100%,9,100%,6,100%,3,100%,3,100%,1,100%,1,100%,0,,1,100%,2,100%,0,,zs53,zs89
190,zs5208_5386,zs8804_8924,2,100%,1,100%,0,,0,,1,100%,0,,0,,0,,0,,0,,0,,zs53,zs89
191,zs5208_5386,zs8804_896702,2,100%,1,100%,0,,0,,1,100%,0,,0,,0,,0,,0,,0,,zs53,zs89
176,zs5208_5377,zs8804_8924,1,100%,1,100%,0,,0,,0,,0,,0,,0,,0,,0,,0,,zs53,zs89


,broker1,broker2,total count,total faster,20201130 count,20201130 faster,20201201 count,20201201 faster,20201202 count,20201202 faster,20201203 count,20201203 faster,20201204 count,20201204 faster,20201207 count,20201207 faster,20201208 count,20201208 faster,20201209 count,20201209 faster,20201210 count,20201210 faster,20201211 count,20201211 faster,b1,b2
448,zs8804_8865,zs9606_9758,1736,12%,0,,0,,0,,23,0%,570,9%,287,17%,252,15%,150,15%,242,10%,212,9%,zs88,zs97
437,zs8804_8856,zs9608_9754,207,77%,45,100%,26,100%,11,100%,17,100%,4,100%,10,70%,8,50%,53,53%,9,67%,24,46%,zs88,zs97


In [19]:
orderLog[(orderLog['server_account'] == 'zs8804_8865') & (orderLog['updateType'] == 0)].groupby(['date', 'sta'])['orderPrice'].size()

date      sta      
20201130  1. staone     201
          2. statwo    2015
20201201  1. staone      94
          2. statwo    1706
20201202  1. staone     117
          2. statwo    1643
20201203  1. staone      93
          2. statwo    2113
20201204  1. staone     488
          2. statwo    2829
20201207  1. staone     257
          2. statwo    2017
20201208  1. staone     269
          2. statwo    1483
20201209  1. staone      19
          2. statwo    1500
20201210  1. staone     265
          2. statwo    1446
20201211  1. staone      73
          2. statwo    1868
Name: orderPrice, dtype: int64

In [31]:
set(orderLog[(orderLog['server_account'] == 'zs8804_8865') & (orderLog['updateType'] == 0) & (orderLog['date'] == 20201130)]['secid'].unique()) - \
set(memb[memb['index_name'] == 'CSI1000']['skey'].unique())

set()

In [18]:
import pymongo
import io
import pandas as pd
import pickle
import datetime
import time
import gzip
import lzma
import pytz
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
memb = read_memb_daily(db1, 'index_memb', 20201130, 20201130)

In [22]:
data1 = pd.read_csv(os.path.join(savePath, 'relativeSpeedBroker_%s_%s_%s.csv'%(startDate, endDate, 'SH')))
def bubble_sort(cur, arr, index):
    arr1 = arr[arr < 0.5]
    index1 = index[arr < 0.5]
    arr2 = arr[arr > 0.5]
    index2 = index[arr > 0.5]
    for i in range(0,len(arr1)):
        for j in range(1,len(arr1)-i):
            if arr1[j]<arr1[j-1]:
                arr1[j-1],arr1[j]=arr1[j],arr1[j-1]
                index1[j-1],index1[j]=index1[j],index1[j-1]
    for i in range(0,len(arr2)):
        for j in range(1,len(arr2)-i):
            if arr2[j]<arr2[j-1]:
                arr2[j-1],arr2[j]=arr2[j],arr2[j-1]
                index2[j-1],index2[j]=index2[j],index2[j-1]
    return cur, list(index1), list(index2)
re = pd.DataFrame()
main = []
bigger = []
smaller = []
for broker, brokerData in data1.groupby(['broker1']):
    kk = bubble_sort(broker, brokerData['total faster'].values, brokerData['broker2'].values)
    main.append(kk[0])
    bigger.append(kk[1])
    smaller.append(kk[2])
re['main'] = main
re['bigger'] = bigger
re['smaller'] = smaller
re['bigger_c'] = re['bigger'].apply(lambda x: len(x))
re['smaller_c'] = re['smaller'].apply(lambda x: len(x))
re['total'] = re['bigger_c'] + re['smaller_c']
display(re[re['total'] <= 2].sort_values(by=['bigger_c']))
display(re[re['total'] > 2].sort_values(by=['bigger_c']))

,main,bigger,smaller,bigger_c,smaller_c,total
10,zt_92,[zt_52],[],1,0,1


,main,bigger,smaller,bigger_c,smaller_c,total
13,zt_96,[],"[zt_52, zt_88, zt_89, zt_53, zt_94, zt_68, zt_...",0,11,11
5,zt_52,[zt_96],"[zt_89, zt_88, zt_94, zt_68, zt_92, zs_97, zt_...",1,12,13
7,zt_68,"[zt_96, zt_52, zt_94]","[zs_97, zt_53, zs_52, zs_53, zs_62, zt_88]",3,6,9
8,zt_88,"[zt_68, zt_96, zt_52]","[zt_53, zt_89, zs_97, zs_89, zs_53, zs_62, zt_95]",3,7,10
9,zt_89,"[zt_96, zt_88, zt_52]","[zt_94, zt_53, zs_97, zs_52, zt_95, zs_89, zs_...",3,8,11
11,zt_94,"[zt_96, zt_52, zt_89]","[zt_68, zs_97, zt_53, zs_53, zs_52, zs_62]",3,6,9
6,zt_53,"[zt_96, zt_52, zt_94, zt_68, zt_88, zt_89]","[zs_97, zt_95, zs_89, zs_53, zs_52, zs_62]",6,6,12
12,zt_95,"[zt_88, zt_96, zt_52, zt_53, zt_89, zs_97]","[zs_52, zs_89]",6,2,8
4,zs_97,"[zt_96, zt_88, zt_52, zt_89, zt_94, zt_53, zt_68]","[zs_52, zt_95, zs_89, zs_53, zs_62]",7,5,12
2,zs_62,"[zs_53, zs_97, zt_52, zt_53, zt_68, zt_88, zt_...",[],8,0,8


In [23]:
data1 = pd.read_csv(os.path.join(savePath, 'relativeSpeedBroker_%s_%s_%s.csv'%(startDate, endDate, 'SZ')))
def bubble_sort(cur, arr, index):
    arr1 = arr[arr < 0.5]
    index1 = index[arr < 0.5]
    arr2 = arr[arr > 0.5]
    index2 = index[arr > 0.5]
    for i in range(0,len(arr1)):
        for j in range(1,len(arr1)-i):
            if arr1[j]<arr1[j-1]:
                arr1[j-1],arr1[j]=arr1[j],arr1[j-1]
                index1[j-1],index1[j]=index1[j],index1[j-1]
    for i in range(0,len(arr2)):
        for j in range(1,len(arr2)-i):
            if arr2[j]<arr2[j-1]:
                arr2[j-1],arr2[j]=arr2[j],arr2[j-1]
                index2[j-1],index2[j]=index2[j],index2[j-1]
    return cur, list(index1), list(index2)
re = pd.DataFrame()
main = []
bigger = []
smaller = []
for broker, brokerData in data1.groupby(['broker1']):
    kk = bubble_sort(broker, brokerData['total faster'].values, brokerData['broker2'].values)
    main.append(kk[0])
    bigger.append(kk[1])
    smaller.append(kk[2])
re['main'] = main
re['bigger'] = bigger
re['smaller'] = smaller
re['bigger_c'] = re['bigger'].apply(lambda x: len(x))
re['smaller_c'] = re['smaller'].apply(lambda x: len(x))
re['total'] = re['bigger_c'] + re['smaller_c']
display(re[re['total'] <= 2].sort_values(by=['bigger_c']))
display(re[re['total'] > 2].sort_values(by=['bigger_c']))

,main,bigger,smaller,bigger_c,smaller_c,total


,main,bigger,smaller,bigger_c,smaller_c,total
11,zs_94,[],"[zs_66, zs_96, zs_72, zs_54, zs_52, zs_88, zs_...",0,15,15
5,zs_66,[zs_94],"[zs_96, zs_52, zs_54, zs_97, zs_53, zs_64, zt_88]",1,7,8
8,zs_88,"[zs_94, zs_96]","[zs_52, zs_54, zs_97, zs_53, zs_68, zs_72, zs_...",2,10,12
12,zs_96,"[zs_94, zs_66]","[zs_88, zs_52, zs_97, zs_54, zs_89, zs_92, zt_...",2,8,10
16,zt_96,"[zs_52, zs_94, zs_97]",[],3,0,3
3,zs_62,"[zs_52, zs_89, zs_97, zs_53]",[],4,0,4
7,zs_72,"[zs_88, zs_52, zs_94, zs_53]","[zs_97, zs_68, zs_89, zt_89]",4,4,8
0,zs_52,"[zs_66, zs_94, zs_96, zs_88]","[zs_54, zs_53, zs_92, zs_97, zs_72, zs_89, zs_...",4,12,16
2,zs_54,"[zs_96, zs_66, zs_94, zs_88, zs_52]","[zs_97, zs_53, zs_92, zt_88, zs_64]",5,5,10
4,zs_64,"[zs_66, zs_94, zs_52, zs_54, zs_53]",[],5,0,5


In [27]:
def color_negative_red(value):
  """
  Colors elements in a dateframe
  green if positive and red if
  negative. Does not color NaN
  values.
  """

  if value < 0:
    color = 'green'
  elif value > 0:
    color = 'red'
  else:
    color = 'black'

  return 'color: %s' % color

thisStartDate = '20201228'
thisEndDate = '20210108'

prevStartDate = '20201214'
prevEndDate = '20201225'

readPath = r'L:\orderLog\result\relativeSpeed'
thisResult = []
for exchg in ['SZ']:
    data = pd.read_csv(os.path.join(readPath, 'relativeSpeedBroker_%s_%s_%s.csv'%(thisStartDate, thisEndDate, exchg)))
    data['exchange'] = exchg
    thisResult += [data]
thisResult = pd.concat(thisResult).reset_index(drop=True)
thisResult['broker1_broker2'] = thisResult[['broker1', 'broker2']].min(axis=1).astype(str) + ' - ' + thisResult[['broker1', 'broker2']].max(axis=1).astype(str)
# thisResult = thisResult.drop_duplicates(subset=['exchange', 'broker1_broker2'], keep='first')
thisResult = thisResult.rename(columns = {'total count': 'thisCount', 'total faster': 'curFaster'})
thisResult = thisResult[['exchange', 'broker1', 'broker2', 'thisCount', 'curFaster']]

prevResult = []
for exchg in ['SZ']:
    data = pd.read_csv(os.path.join(readPath, 'relativeSpeedBroker_%s_%s_%s.csv'%(prevStartDate, prevEndDate, exchg)))
    data['exchange'] = exchg
    prevResult += [data]
prevResult = pd.concat(prevResult).reset_index(drop=True)
prevResult['broker1_broker2'] = prevResult[['broker1', 'broker2']].min(axis=1).astype(str) + ' - ' + prevResult[['broker1', 'broker2']].max(axis=1).astype(str)
# prevResult = prevResult.drop_duplicates(subset=['exchange', 'broker1_broker2'], keep='first')
prevResult = prevResult.rename(columns = {'total count': 'prevCount', 'total faster': 'prevFaster'})
prevResult = prevResult[['exchange', 'broker1', 'broker2', 'prevCount', 'prevFaster']]

checkResult = pd.merge(thisResult, prevResult, how='outer', on=['exchange', 'broker1', 'broker2'], validate='one_to_one')
checkResult['speedDif'] = checkResult['curFaster'] - checkResult['prevFaster']


# pp = checkResult[(((checkResult['broker1'] == 'zs_52')) & (checkResult['exchange'] == 'SZ'))]
# pp = pp.groupby(['exchange', 'broker1', 'broker2'])[['prevCount', 'prevFaster', 'thisCount', 'curFaster','speedDif']].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     pp[col] = pp[col].apply(lambda x: '%.0f%%'%(x*100) if ~np.isnan(x) else x)
# from IPython.display import display, HTML
# display(HTML(pp.to_html()))

# pp = checkResult[(((checkResult['broker1'] == 'zs_54')) & (checkResult['exchange'] == 'SZ'))]
# pp = pp.groupby(['exchange', 'broker1', 'broker2'])[['prevCount', 'prevFaster', 'thisCount', 'curFaster','speedDif']].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     pp[col] = pp[col].apply(lambda x: '%.0f%%'%(x*100) if ~np.isnan(x) else x)
# from IPython.display import display, HTML
# display(HTML(pp.to_html()))

# pp = checkResult[(((checkResult['broker1'] == 'zs_53')) & (checkResult['exchange'] == 'SZ'))]
# pp = pp.groupby(['exchange', 'broker1', 'broker2'])[['prevCount', 'prevFaster', 'thisCount', 'curFaster','speedDif']].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     pp[col] = pp[col].apply(lambda x: '%.0f%%'%(x*100) if ~np.isnan(x) else x)
# from IPython.display import display, HTML
# display(HTML(pp.to_html()))

# pp = checkResult[(((checkResult['broker1'] == 'zs_88')) & (checkResult['exchange'] == 'SZ'))]
# pp = pp.groupby(['exchange', 'broker1', 'broker2'])[['prevCount', 'prevFaster', 'thisCount', 'curFaster','speedDif']].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     pp[col] = pp[col].apply(lambda x: '%.0f%%'%(x*100) if ~np.isnan(x) else x)
# from IPython.display import display, HTML
# display(HTML(pp.to_html()))

# pp = checkResult[(((checkResult['broker1'] == 'zs_66')) & (checkResult['exchange'] == 'SZ'))]
# pp = pp.groupby(['exchange', 'broker1', 'broker2'])[['prevCount', 'prevFaster', 'thisCount', 'curFaster','speedDif']].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     pp[col] = pp[col].apply(lambda x: '%.0f%%'%(x*100) if ~np.isnan(x) else x)
# from IPython.display import display, HTML
# display(HTML(pp.to_html()))

# check1 = checkResult[(checkResult["broker1"].isin(["zt_52", "zt_88"])) & (checkResult["exchange"] == "SH")].groupby(['exchange', 'broker1', 'broker2'])[['prevFaster','curFaster','speedDif',"prevCount", "thisCount"]].first()
# for col in ['curFaster', 'prevFaster', 'speedDif']:
#     check1[col] = check1[col].apply(lambda x: '%.0f%%'%(x*100))
# for col in ["prevCount"]:
#     check1[col] = check1[col].fillna(0)
#     check1[col] = check1[col].astype(int)
# display(HTML(check1.to_html()))

# checkResult = checkResult[checkResult["broker1"] == "zs_52"].groupby(['exchange', 'broker1', 'broker2'])[['prevFaster','curFaster','speedDif']].first()
# checkResult = checkResult[(abs(checkResult['speedDif']) > 0.15) & (checkResult["broker1"] == "zs_52")].groupby(['exchange', 'broker1', 'broker2'])[['prevFaster','curFaster','speedDif']].first()
# checkResult = checkResult[(checkResult["broker1"] == "zs_54") | (checkResult["broker1"] == "zs_52") | (checkResult["broker1"] == "zs_97")
#                          | (checkResult["broker1"] == "zs_92")].groupby(['exchange', 'broker1', 'broker2'])[['prevFaster','curFaster','speedDif']].first()


checkResult = checkResult[(abs(checkResult['speedDif']) > 0.15) & (checkResult["thisCount"] > 100) & (checkResult['prevCount'] > 100)].groupby(['exchange', 'broker1', 'broker2'])[['prevFaster','curFaster','speedDif']].first()
# checkResult = checkResult[(checkResult['broker1'] == 'zt_89') & (checkResult["thisCount"] > 100) & (checkResult['prevCount'] > 100) ].groupby(['exchange', 'broker1', 'broker2']).first().sort_values(by='speedDif')
for col in ['curFaster', 'prevFaster', 'speedDif']:
#     checkResult[col] = checkResult[col].apply(lambda x: '%.0f%%'%(x*100))
    checkResult[col] = checkResult[col]*100

from IPython.display import display, HTML
# display(HTML(checkResult.to_html()))
display(checkResult.style.format({"prevFaster": "{:0,.0f}%", "curFaster": "{:0,.0f}%","speedDif": "{:0,.0f}%"}).applymap(color_negative_red, subset=['speedDif']))

In [59]:
orderLog[orderLog['colo'].str[:5] == 'zt_52']['colo'].unique()

array(['zt_52_05', 'zt_52_07', 'zt_52_04', 'zt_52_10'], dtype=object)

In [6]:
checkResult['b1'] = checkResult['broker1'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
checkResult['b2'] = checkResult['broker2'].apply(lambda x: x.split('_')[0][:2] + x.split('_')[1][:2])
checkResult[(checkResult['b1'] == 'zs66') & (checkResult['b2'] == 'zs96') & (checkResult["thisCount"] > 100)].groupby(['exchange', 'broker1', 'broker2']).first()

thisCount  curFaster  prevCount  prevFaster  \
exchange broker1     broker2                                                    
SZ       zs6601_6623 zs9608_9685      768.0   0.430990     1079.0    0.342910   
         zs6601_6627 zs9608_9685      694.0   0.353026     1101.0    0.318801   
         zs6601_6683 zs9608_9685     2088.0   0.625958     2511.0    0.576663   

                                  speedDif    b1    b2  
exchange broker1     broker2                            
SZ       zs6601_6623 zs9608_9685  0.088079  zs66  zs96  
         zs6601_6627 zs9608_9685  0.034225  zs66  zs96  
         zs6601_6683 zs9608_9685  0.049295  zs66  zs96

In [74]:
# tickToMBD
checkLog = orderLog[~orderLog['start_time'].isnull()]
checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog.reset_index(drop=True)
checkLog['ordering'] = checkLog.index
checkLog['time_diff'] = checkLog['caa_orderLog'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]
ol = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '2. statwo'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol1 = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol2 = pd.merge(checkLog[checkLog['sta'] == '2. statwo'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
checkLog1 = checkLog[(checkLog['sta'] == '1. staone') & (~checkLog['ordering'].isin(ol)) & (~checkLog['ordering'].isin(ol1))]
checkLog2 = checkLog[(checkLog['sta'] == '2. statwo') & (~checkLog['ordering'].isin(ol2))]
checkLog3 = checkLog[checkLog['sta'] == '3. sta300']

from IPython.display import display, HTML
re1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))


In [6]:
# SZE order speed compared with market
startTm = datetime.datetime.now()

statsData1 = {}
statsData2 = {}
statsData3 = {}
for col in ['account', 'date', "orderNtl", "tradeNtl", "gp", 'secid', 'vai', 'side', 'our ntl', 'market orders', 'market ntl', 'front orders', 'front ntl', 'statusLs', "TradePriceLs", "TradeQtyLs"]:
    statsData1[col] = []
    statsData2[col] = []
    statsData3[col] = []

startDate = '20201130'
endDate = '20201130'

readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'marketPos1_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    thisDate = os.path.basename(path).split('.')[0].split('_')[1]
    data = pd.read_pickle(path)
    data['date'] = thisDate
    checkData += [data]
checkData = pd.concat(checkData).reset_index(drop=True)   

checkData["firstclock"] = checkData.groupby(["date", "group"])["clockAtArrival"].transform("first")
checkData = checkData[checkData["firstclock"] != checkData["clockAtArrival"]]

checkData['brokerLs'] = np.where(checkData['brokerLs'].isnull(), ' ', checkData['brokerLs'])
checkData['brokerLs'] = checkData['brokerLs'].apply(lambda x: x.split(','))
checkData['firstBroker'] = checkData['brokerLs'].apply(lambda x: x[0])
checkData['sta'] = np.where(checkData['mrstaat'] == 1000, '1. staone', np.where(
checkData['mrstaat'] == 3000, '2. statwo', np.where(
checkData['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))

def f(x):
    n = len(x[0])
    m = 0
    for i in range(n):
        m = m + x[0][i] * x[1][i] / 10000
    return m
    
checkData["tradeNtl"] = checkData[["TradePriceLs", "TradeQtyLs"]].apply(f, axis=1)

for date, dateData in checkData.groupby(['date']):
    print(date)
    for group, groupData in dateData.groupby(['group']):
        groupData = groupData.reset_index(drop=True)
        ### ignore cases where brokerNum >= 2 (around 2.5%, 7% data)
        if groupData[groupData['brokerNum'] >= 2].shape[0] == 0:
            groupData['index'] = groupData.index.values
            groupData['lastIx'] = groupData.groupby(['firstBroker'])['index'].transform('last')
            groupData['isOurs'] = np.where((groupData['index'] == groupData['lastIx']) & (groupData['brokerNum'] == 1), 1, 0)
            brokerLs = groupData[groupData['brokerNum'] == 1]['firstBroker'].unique()
            for broker in brokerLs:
                if groupData[(groupData["firstBroker"] == broker) & (groupData["isOurs"] == 1)]['sta'].values == ['2. statwo']:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData1['account'].append(broker)
                    statsData1['date'].append(date)
                    statsData1['gp'].append(group)
                    statsData1['secid'].append(groupData['secid'].values[0])
                    statsData1['vai'].append(groupData['vai'].values[0])
                    statsData1['side'].append(groupData['Side'].values[0])
                    statsData1['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData1['market orders'].append(totalOrders)
                    statsData1['market ntl'].append(totalSize)
                    statsData1['front orders'].append(frontData.shape[0])
                    statsData1['front ntl'].append(frontSize)
                    statsData1['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData1['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData1['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData1['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData1['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
                
                elif groupData[(groupData["firstBroker"] == broker) & (groupData["isOurs"] == 1)]['sta'].values == ['1. staone']:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData2['account'].append(broker)
                    statsData2['date'].append(date)
                    statsData2['gp'].append(group)
                    statsData2['secid'].append(groupData['secid'].values[0])
                    statsData2['vai'].append(groupData['vai'].values[0])
                    statsData2['side'].append(groupData['Side'].values[0])
                    statsData2['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData2['market orders'].append(totalOrders)
                    statsData2['market ntl'].append(totalSize)
                    statsData2['front orders'].append(frontData.shape[0])
                    statsData2['front ntl'].append(frontSize)
                    statsData2['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData2['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData2['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData2['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData2['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
                
                else:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData3['account'].append(broker)
                    statsData3['date'].append(date)
                    statsData3['gp'].append(group)
                    statsData3['secid'].append(groupData['secid'].values[0])
                    statsData3['vai'].append(groupData['vai'].values[0])
                    statsData3['side'].append(groupData['Side'].values[0])
                    statsData3['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData3['market orders'].append(totalOrders)
                    statsData3['market ntl'].append(totalSize)
                    statsData3['front orders'].append(frontData.shape[0])
                    statsData3['front ntl'].append(frontSize)
                    statsData3['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData3['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData3['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData3['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData3['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
statsData1 = pd.DataFrame(statsData1)
statsData1['exchange'] = np.where(statsData1['secid'] >= 2000000, 'SZE', 'SSE')
statsData1["broker"] = statsData1["account"].str[:5]
statsData1['count'] = statsData1.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

statsData2 = pd.DataFrame(statsData2)
statsData2['exchange'] = np.where(statsData2['secid'] >= 2000000, 'SZE', 'SSE')
statsData2["broker"] = statsData2["account"].str[:5]
statsData2['count'] = statsData2.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

statsData3 = pd.DataFrame(statsData3)
statsData3['exchange'] = np.where(statsData3['secid'] >= 2000000, 'SZE', 'SSE')
statsData3["broker"] = statsData3["account"].str[:5]
statsData3['count'] = statsData3.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

print(datetime.datetime.now() - startTm)

20201130
0:08:44.944117


In [ ]:
summaryData = statsData1.copy()
summaryData['count'] = summaryData.groupby(['broker', 'exchange'])['secid'].transform('count')
summaryData['# of other orders'] = np.where(summaryData['market orders'] == 0, '0',
                                   np.where(summaryData['market orders'] <= 2, '1-2',
                                   np.where(summaryData['market orders'] <= 5, '3-5', '>= 6')))
summaryData['% count'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['secid'].transform('count')
summaryData['% count'] = summaryData['% count'] / summaryData['count']
summaryData['notional of other orders'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['market ntl'].transform('mean')
summaryData['# of total orders'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['market orders'].transform('sum')
summaryData['% orders in front'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['front orders'].transform('sum')
summaryData['% orders in front'] = summaryData['% orders in front'] / summaryData['# of total orders']

summaryData['notional of other orders clip'] = summaryData['market ntl'].clip(0, 5000000)
summaryData['notional of other orders clip'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['notional of other orders clip'].transform('mean')
summaryData['% notional in front'] = summaryData['front ntl'].clip(0, 5000000)
summaryData['% notional in front'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['% notional in front'].transform('mean')
summaryData['% notional in front'] = summaryData['% notional in front'] / summaryData['notional of other orders clip']

savePath = r'L:\orderLog\result\marketSpeed'
a = summaryData.groupby(['broker', 'exchange', 'count', '# of other orders'])[['% count', 'notional of other orders', '% orders in front', '% notional in front']].mean().reset_index()

summaryData = summaryData.groupby(['broker', 'exchange', 'count', '# of other orders'])[['% count', 'notional of other orders', '% orders in front', '% notional in front']].mean()
for col in [ 'notional of other orders']:
    summaryData[col] = summaryData[col].astype('int64')
for col in ['% count', '% orders in front', '% notional in front']:
    summaryData[col] = summaryData[col].fillna(0)
    summaryData[col] = summaryData[col].apply(lambda x: '%.0f%%'%(x*100))
from IPython.display import display, HTML
display(HTML(summaryData.to_html()))

In [75]:
# fill rate
orderLog['Price'] = orderLog['orderPrice'].apply(lambda x: round(x*100, 0))
orderLog['firstClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['clockDif'] = orderLog['clockAtArrival'] - orderLog['firstClock']
orderLog['isImmediate'] = np.where(orderLog['secid'] >= 2000000,
                          np.where(orderLog['colo_broker'] == 'zs_62', 
                                   np.where(orderLog['clockDif'] <= 1000000, 1, 0),
                                   np.where(orderLog['colo_broker'].isin(['zt_88', 'zt_89', 'zt_96']),
                                           np.where(orderLog['clockDif'] <= 50000, 1, 0),
                                           np.where(orderLog['clockDif'] <= 20000, 1, 0))),
                          np.where(orderLog['clockDif'] <= 1000000, 1, 0))
orderLog['orderNtl'] = orderLog['Price'] * orderLog['absOrderSize'] / 100

# SZE triggered by mbd using staone
display(orderLog[(orderLog["updateType"] == 0) & (orderLog["isMsg"] == 1) & (orderLog["secid"] >= 2000000)].shape[0] 
/ orderLog[(orderLog["updateType"] == 0) & (orderLog["secid"] >= 2000000)].shape[0])

checkData = orderLog[orderLog['isImmediate'] == 1].reset_index(drop=True)
checkData['exchange'] = np.where(checkData['secid'] >= 2000000, 'SZE', 'SSE')
checkData['maxFilled'] = checkData.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
checkData['immedFillNtl'] = checkData['maxFilled'] * checkData['Price'] / 100
checkData = checkData[checkData['updateType'] == 0]

checkData = checkData[(checkData["exchange"] == "SZE") & (checkData['sta'] == '1. staone') & (checkData["isMsg"] == 1)]

immedFillData = checkData.groupby(['exchange', 'colo_broker'])[['orderNtl', 'immedFillNtl']].sum().reset_index()
immedFillData = immedFillData.rename(columns={'colo_broker': 'broker'})
immedFillData['immedFillPerc'] = immedFillData['immedFillNtl'] / immedFillData['orderNtl']
immedFillData = immedFillData[['exchange', 'broker', 'immedFillPerc']]


checkData['fillStatus'] = np.where(checkData['maxFilled'] == 0, 0,
                          np.where(checkData['maxFilled'] < checkData['absOrderSize'], 1, 2))
statsData = checkData.groupby(['exchange', 'colo_broker', 'fillStatus'])['secid'].count().to_frame().reset_index()
statsData.columns = ['exchange', 'broker', 'fillStatus', 'count']
statsData = statsData.reset_index()
statsData['# of orders'] = statsData.groupby(['exchange'])['count'].transform('sum')
statsData['% of orders'] = statsData.groupby(['broker', 'exchange'])['count'].transform('sum')
statsData['percent'] = statsData['count'] / statsData['% of orders']
statsData['% of orders'] = statsData['% of orders'] / statsData['# of orders']
saveCols = ['broker', 'exchange', 'percent']
noFillData = statsData[statsData['fillStatus'] == 0][saveCols].reset_index(drop=True)
noFillData = noFillData.rename(columns={'percent': 'no fill'})
partialFillData = statsData[statsData['fillStatus'] == 1][saveCols].reset_index(drop=True)
partialFillData = partialFillData.rename(columns={'percent': 'partial fill'})
fullFillData = statsData[statsData['fillStatus'] == 2][saveCols].reset_index(drop=True)
fullFillData = fullFillData.rename(columns={'percent': 'full fill'})


orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')

o1 = orderLog[(orderLog["exchange"] == "SZE") & (orderLog['sta'] == '1. staone') & (orderLog["isMsg"] == 1)]

fillRateData = o1.groupby(['exchange', 'colo_broker', 'accCode', 'secid', 'order'])[['absOrderSizeCumFilled', 'absOrderSize', 'orderPrice']].max().reset_index()
fillRateData = fillRateData.rename(columns={'colo_broker': 'broker'})
fillRateData['orderNotional'] = fillRateData['absOrderSize']*fillRateData['orderPrice']
fillRateData['fillNotional'] = fillRateData['absOrderSizeCumFilled']*fillRateData['orderPrice']
fillRateData['totalNotional'] = fillRateData.groupby(['exchange', 'broker'])['orderNotional'].transform('sum')
fillRateData['fillNotional'] = fillRateData.groupby(['exchange', 'broker'])['fillNotional'].transform('sum')
fillRateData['fillPerc'] = fillRateData['fillNotional'] / fillRateData['totalNotional']
fillRateData = fillRateData.groupby(['exchange', 'broker'])['fillPerc'].mean().reset_index()

statsData = statsData.groupby(['exchange', 'broker', '# of orders'])['% of orders'].first().reset_index()
statsData = pd.merge(statsData, noFillData, how='outer', on=['broker', 'exchange'], validate='one_to_one')
statsData = pd.merge(statsData, partialFillData, how='outer', on=['broker', 'exchange'], validate='one_to_one')
statsData = pd.merge(statsData, fullFillData, how='outer', on=['broker', 'exchange'], validate='one_to_one')
statsData = pd.merge(statsData, fillRateData, how='outer', on=['broker', 'exchange'], validate='one_to_one')
statsData = pd.merge(statsData, immedFillData, how='outer', on=['broker', 'exchange'], validate='one_to_one')


statsData = statsData.groupby(['exchange', '# of orders', 'broker'])[['% of orders', 'full fill', 'partial fill', 'no fill', 'fillPerc', 'immedFillPerc']].first()

savePath = r'L:\orderLog\result\fillRate'
statsData.reset_index().to_csv(os.path.join(savePath, 'fillRate2_%s_%s.csv'%(startDate, endDate)), index=False)


from IPython.display import display, HTML
for col in ['% of orders', 'no fill', 'partial fill', 'full fill', 'fillPerc', 'immedFillPerc']:
    statsData[col] = statsData[col].fillna(0)
    statsData[col] = statsData[col].apply(lambda x: '%.0f%%'%(x*100))
display(HTML(statsData.to_html()))

0.9913132875272088

In [76]:
# SSE staone
checkData = orderLog[orderLog['isImmediate'] == 1].reset_index(drop=True)
checkData['exchange'] = np.where(checkData['secid'] >= 2000000, 'SZE', 'SSE')
checkData['maxFilled'] = checkData.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
checkData['immedFillNtl'] = checkData['maxFilled'] * checkData['Price'] / 100
checkData = checkData[checkData['updateType'] == 0]


checkData = checkData[(checkData["exchange"] == "SSE") & (checkData['sta'] == '1. staone')]

immedFillData = checkData.groupby(['exchange', 'colo_broker', 'isMsg'])[['orderNtl', 'immedFillNtl']].sum().reset_index()
immedFillData = immedFillData.rename(columns={'colo_broker': 'broker'})
immedFillData['immedFillPerc'] = immedFillData['immedFillNtl'] / immedFillData['orderNtl']
immedFillData = immedFillData[['exchange', 'broker', 'immedFillPerc', 'isMsg']]


checkData['fillStatus'] = np.where(checkData['maxFilled'] == 0, 0,
                          np.where(checkData['maxFilled'] < checkData['absOrderSize'], 1, 2))
statsData = checkData.groupby(['exchange', 'colo_broker', 'isMsg', 'fillStatus'])['secid'].count().to_frame().reset_index()
statsData.columns = ['exchange', 'broker', 'isMsg', 'fillStatus', 'count']
statsData = statsData.reset_index()
statsData['# of orders'] = statsData.groupby(['exchange'])['count'].transform('sum')
statsData['% of orders'] = statsData.groupby(['broker', 'exchange', "isMsg"])['count'].transform('sum')
statsData['percent'] = statsData['count'] / statsData['% of orders']
statsData['% of orders'] = statsData['% of orders'] / statsData['# of orders']
saveCols = ['broker', 'exchange', 'isMsg', 'percent']
noFillData = statsData[statsData['fillStatus'] == 0][saveCols].reset_index(drop=True)
noFillData = noFillData.rename(columns={'percent': 'no fill'})
partialFillData = statsData[statsData['fillStatus'] == 1][saveCols].reset_index(drop=True)
partialFillData = partialFillData.rename(columns={'percent': 'partial fill'})
fullFillData = statsData[statsData['fillStatus'] == 2][saveCols].reset_index(drop=True)
fullFillData = fullFillData.rename(columns={'percent': 'full fill'})


orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')

o1 = orderLog[(orderLog["exchange"] == "SSE") & (orderLog['sta'] == '1. staone')]



fillRateData = o1.groupby(['exchange', 'colo_broker', 'accCode', 'secid', 'order', 'isMsg'])[['absOrderSizeCumFilled', 'absOrderSize', 'orderPrice']].max().reset_index()
fillRateData = fillRateData.rename(columns={'colo_broker': 'broker'})
fillRateData['orderNotional'] = fillRateData['absOrderSize']*fillRateData['orderPrice']
fillRateData['fillNotional'] = fillRateData['absOrderSizeCumFilled']*fillRateData['orderPrice']
fillRateData['totalNotional'] = fillRateData.groupby(['exchange', 'broker', "isMsg"])['orderNotional'].transform('sum')
fillRateData['fillNotional'] = fillRateData.groupby(['exchange', 'broker', "isMsg"])['fillNotional'].transform('sum')
fillRateData['fillPerc'] = fillRateData['fillNotional'] / fillRateData['totalNotional']
fillRateData = fillRateData.groupby(['exchange', 'broker', "isMsg"])['fillPerc'].mean().reset_index()

statsData = statsData.groupby(['exchange', 'broker', 'isMsg', '# of orders'])['% of orders'].first().reset_index()
statsData = pd.merge(statsData, noFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, partialFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, fullFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, fillRateData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, immedFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')


# statsData = statsData.groupby(['exchange', '# of orders', 'broker'])[['% of orders', 'full fill', 'partial fill', 'no fill', 'fillPerc', 'immedFillPerc']].first()
statsData['isMsg'] = statsData['isMsg'].astype("int")
statsData = statsData.groupby(['exchange', '# of orders','broker', 'isMsg' ])[['% of orders', 'full fill', 'partial fill', 'no fill', 'fillPerc', 'immedFillPerc']].first()

savePath = r'L:\orderLog\result\fillRate'
statsData.reset_index().to_csv(os.path.join(savePath, 'fillRate1_%s_%s.csv'%(startDate, endDate)), index=False)


from IPython.display import display, HTML
for col in ['% of orders', 'no fill', 'partial fill', 'full fill', 'fillPerc', 'immedFillPerc']:
    statsData[col] = statsData[col].fillna(0)
    statsData[col] = statsData[col].apply(lambda x: '%.0f%%'%(x*100))
display(HTML(statsData.to_html()))

In [77]:
# statwo
checkData = orderLog[orderLog['isImmediate'] == 1].reset_index(drop=True)
checkData['exchange'] = np.where(checkData['secid'] >= 2000000, 'SZE', 'SSE')
checkData['maxFilled'] = checkData.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
checkData['immedFillNtl'] = checkData['maxFilled'] * checkData['Price'] / 100
checkData = checkData[checkData['updateType'] == 0]

checkData = pd.concat([checkData[(checkData["sta"] == '2. statwo') & (checkData["exchange"] == 'SSE')], 
                       checkData[(checkData["sta"] == '2. statwo') & (checkData["exchange"] == 'SZE') &
                                (checkData["isMsg"] == 1)]])


immedFillData = checkData.groupby(['exchange', 'colo_broker', 'isMsg'])[['orderNtl', 'immedFillNtl']].sum().reset_index()
immedFillData = immedFillData.rename(columns={'colo_broker': 'broker'})
immedFillData['immedFillPerc'] = immedFillData['immedFillNtl'] / immedFillData['orderNtl']
immedFillData = immedFillData[['exchange', 'broker', 'immedFillPerc', 'isMsg']]


checkData['fillStatus'] = np.where(checkData['maxFilled'] == 0, 0,
                          np.where(checkData['maxFilled'] < checkData['absOrderSize'], 1, 2))
statsData = checkData.groupby(['exchange', 'colo_broker', 'isMsg', 'fillStatus'])['secid'].count().to_frame().reset_index()
statsData.columns = ['exchange', 'broker', 'isMsg', 'fillStatus', 'count']
statsData = statsData.reset_index()
statsData['# of orders'] = statsData.groupby(['exchange'])['count'].transform('sum')
statsData['% of orders'] = statsData.groupby(['broker', 'exchange', "isMsg"])['count'].transform('sum')
statsData['percent'] = statsData['count'] / statsData['% of orders']
statsData['% of orders'] = statsData['% of orders'] / statsData['# of orders']
saveCols = ['broker', 'exchange', 'isMsg', 'percent']
noFillData = statsData[statsData['fillStatus'] == 0][saveCols].reset_index(drop=True)
noFillData = noFillData.rename(columns={'percent': 'no fill'})
partialFillData = statsData[statsData['fillStatus'] == 1][saveCols].reset_index(drop=True)
partialFillData = partialFillData.rename(columns={'percent': 'partial fill'})
fullFillData = statsData[statsData['fillStatus'] == 2][saveCols].reset_index(drop=True)
fullFillData = fullFillData.rename(columns={'percent': 'full fill'})


orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')

o1 = pd.concat([orderLog[(orderLog["sta"] == '2. statwo') & (orderLog["exchange"] == 'SSE')], orderLog[
    ((orderLog["sta"] == '2. statwo') & (orderLog["exchange"] == 'SZE') & (orderLog["isMsg"] == 1))
]])
# o1 = orderLog[(orderLog["exchange"] == "SSE") & (orderLog["ars"] != 21)]



fillRateData = o1.groupby(['exchange', 'colo_broker', 'accCode', 'secid', 'order', 'isMsg'])[['absOrderSizeCumFilled', 'absOrderSize', 'orderPrice']].max().reset_index()
fillRateData = fillRateData.rename(columns={'colo_broker': 'broker'})
fillRateData['orderNotional'] = fillRateData['absOrderSize']*fillRateData['orderPrice']
fillRateData['fillNotional'] = fillRateData['absOrderSizeCumFilled']*fillRateData['orderPrice']
fillRateData['totalNotional'] = fillRateData.groupby(['exchange', 'broker', "isMsg"])['orderNotional'].transform('sum')
fillRateData['fillNotional'] = fillRateData.groupby(['exchange', 'broker', "isMsg"])['fillNotional'].transform('sum')
fillRateData['fillPerc'] = fillRateData['fillNotional'] / fillRateData['totalNotional']
fillRateData = fillRateData.groupby(['exchange', 'broker', "isMsg"])['fillPerc'].mean().reset_index()

statsData = statsData.groupby(['exchange', 'broker', 'isMsg', '# of orders'])['% of orders'].first().reset_index()
statsData = pd.merge(statsData, noFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, partialFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, fullFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, fillRateData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')
statsData = pd.merge(statsData, immedFillData, how='outer', on=['broker', 'exchange', 'isMsg'], validate='one_to_one')


# statsData = statsData.groupby(['exchange', '# of orders', 'broker'])[['% of orders', 'full fill', 'partial fill', 'no fill', 'fillPerc', 'immedFillPerc']].first()
statsData['isMsg'] = statsData['isMsg'].astype("int")
statsData = statsData.groupby(['exchange', '# of orders','broker', 'isMsg' ])[['% of orders', 'full fill', 'partial fill', 'no fill', 'fillPerc', 'immedFillPerc']].first()

savePath = r'L:\orderLog\result\fillRate'
statsData.reset_index().to_csv(os.path.join(savePath, 'fillRate3_%s_%s.csv'%(startDate, endDate)), index=False)


from IPython.display import display, HTML
for col in ['% of orders', 'no fill', 'partial fill', 'full fill', 'fillPerc', 'immedFillPerc']:
    statsData[col] = statsData[col].fillna(0)
    statsData[col] = statsData[col].apply(lambda x: '%.0f%%'%(x*100))
display(HTML(statsData.to_html()))

In [102]:
thisStartDate = '20201228'
thisEndDate = '20210108'

prevStartDate = '20201214'
prevEndDate = '20201225'

def color_negative_red(value):
  """
  Colors elements in a dateframe
  green if positive and red if
  negative. Does not color NaN
  values.
  """

  if value < 0:
    color = 'green'
  elif value > 0:
    color = 'red'
  else:
    color = 'black'

  return 'color: %s' % color

readPath = r'L:\orderLog\result\fillRate'
thisResult = pd.read_csv(os.path.join(readPath, 'fillRate1_%s_%s.csv'%(thisStartDate, thisEndDate)))
thisResult = thisResult.rename(columns={'% of orders': 'thisCountPerc', 'full fill': 'thisFullFill',
                                        'partial fill': 'thisPartialFill', 'no fill': 'thisNoFill',
                                        'fillPerc': 'thisFillPerc', 'immedFillPerc': 'thisImmedFillPerc'})


prevResult = pd.read_csv(os.path.join(readPath, 'fillRate1_%s_%s.csv'%(prevStartDate, prevEndDate)))
prevResult = prevResult.rename(columns={'% of orders': 'prevCountPerc', 'full fill': 'prevFullFill',
                                        'partial fill': 'prevPartialFill', 'no fill': 'prevNoFill',
                                        'fillPerc': 'prevFillPerc', 'immedFillPerc': 'prevImmedFillPerc'})

checkResult = pd.merge(thisResult, prevResult, how='left', on=['exchange', 'broker', "isMsg"], validate='one_to_one')
checkResult = checkResult.rename(columns={"# of orders_x": "this order", "# of orders_y": "prev order"})
checkResult['countPercDif'] = checkResult['thisCountPerc'] - checkResult['prevCountPerc']
checkResult['fullFillDif'] = checkResult['thisFullFill'] - checkResult['prevFullFill']
checkResult['partialFillDif'] = checkResult['thisPartialFill'] - checkResult['prevPartialFill']
checkResult['noFillDif'] = checkResult['thisNoFill'] - checkResult['prevNoFill']
checkResult['fillPercDif'] = checkResult['thisFillPerc'] - checkResult['prevFillPerc']
checkResult['immedFillPercDif'] = checkResult['thisImmedFillPerc'] - checkResult['prevImmedFillPerc']
re1 = checkResult[abs(checkResult['fullFillDif']) > 0.1]
re2 = checkResult[abs(checkResult['noFillDif']) > 0.1]
for col in ['prevFullFill', 'thisFullFill', 'fullFillDif',  'prevNoFill', 'thisNoFill', 'noFillDif']:
    re1[col] = re1[col].apply(lambda x: '%.0f%%'%(x*100))
    re2[col] = re2[col].apply(lambda x: '%.0f%%'%(x*100))

# display(checkResult[abs(checkResult['countPercDif']) > 0.1][['exchange', 'broker', 'isMsg', 'prevCountPerc', 'thisCountPerc', 'countPercDif']])
# display(re1[['exchange', 'broker', 'isMsg', 'prevFullFill', 'thisFullFill', 'fullFillDif']])
# display(checkResult[abs(checkResult['partialFillDif']) > 0.1][['exchange', 'broker', 'isMsg', 'prevPartialFill', 'thisPartialFill', 'partialFillDif']])
# display(re2[['exchange', 'broker', 'isMsg', 'prevNoFill', 'thisNoFill', 'noFillDif']])
# display(checkResult[abs(checkResult['fillPercDif']) > 0.1][['exchange', 'broker', 'prevFillPerc', 'thisFillPerc', 'fillPercDif']])
# display(checkResult[abs(checkResult['immedFillPercDif']) > 0.1][['exchange', 'broker', 'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']])
a = checkResult[(abs(checkResult['fillPercDif']) > 0.1)].reset_index(drop=True)
# exchangeTurnover, check trade notional in each broker
# a = pd.merge(a, exchangeTurnover.reset_index()[['exchange', 'broker', 'turnover']], how='left', on=['exchange', 'broker'], validate='many_to_one')
a["thisCount"] = a["this order"] * a["thisCountPerc"]
a["prevCount"] = a["prev order"] * a["prevCountPerc"]
a = a[['exchange', 'broker', 'isMsg', 'thisCount',"prevCount", 'prevFillPerc', 'thisFillPerc', 'fillPercDif',  'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']]
for col in ['prevFillPerc', 'thisFillPerc', 'fillPercDif',  'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']:
#     a[col] = a[col].apply(lambda x: '%.0f%%'%(x*100))
    a[col] = a[col] * 100
a["thisCount"] = a["thisCount"].round(0).astype(int)
a["prevCount"] = a["prevCount"].round(0).astype(int)
display(a[(a["thisCount"] > 100) & (a["prevCount"] > 100)])

display(pd.concat([a[(a["thisCount"] > 100) & (a["prevCount"] > 100) & (a['exchange'] == 'SSE') & (a['broker'].str[:2] == 'zt')][['exchange', 'broker', 'isMsg', 'prevFillPerc', 'thisFillPerc',  'fillPercDif', 'prevImmedFillPerc', 'thisImmedFillPerc']], \
                 a[(a["thisCount"] > 100) & (a["prevCount"] > 100) & (a['exchange'] == 'SZE') & (a['broker'].str[:2] == 'zs')][['exchange', 'broker', 'isMsg', 'prevFillPerc', 'thisFillPerc',  'fillPercDif', 'prevImmedFillPerc', 'thisImmedFillPerc']]]).style.format({"prevFillPerc": "{:0,.0f}%", "thisFillPerc": "{:0,.0f}%","fillPercDif": "{:0,.0f}%","prevImmedFillPerc": "{:0,.0f}%",\
                  "thisImmedFillPerc": "{:0,.0f}%"}).applymap(color_negative_red, subset=['fillPercDif']))
# pd.concat([re1[['exchange', 'broker', 'prevFullFill', 'thisFullFill', 'fullFillDif','prevNoFill', 'thisNoFill', 'noFillDif']], 
#            re2[['exchange', 'broker', 'prevFullFill', 'thisFullFill', 'fullFillDif','prevNoFill', 'thisNoFill', 'noFillDif']]]).drop_duplicates()

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,exchange,broker,isMsg,thisCount,prevCount,prevFillPerc,thisFillPerc,fillPercDif,prevImmedFillPerc,thisImmedFillPerc,immedFillPercDif
0,SSE,zs_53,1,210,264,43.345148,60.821996,17.476848,19.207481,38.856148,19.648667
2,SSE,zs_97,1,663,485,82.028070,70.450743,-11.577327,70.385721,54.199872,-16.185849


,exchange,broker,isMsg,prevFillPerc,thisFillPerc,fillPercDif,prevImmedFillPerc,thisImmedFillPerc


In [103]:
### here we need exchangeTurnover data in part 1 Order Size

thisStartDate = '20201228'
thisEndDate = '20210108'

prevStartDate = '20201214'
prevEndDate = '20201225'

readPath = r'L:\orderLog\result\fillRate'
thisResult = pd.read_csv(os.path.join(readPath, 'fillRate2_%s_%s.csv'%(thisStartDate, thisEndDate)))
# thisResult = thisResult.drop(columns=['# of orders'])
thisResult = thisResult.rename(columns={'% of orders': 'thisCountPerc', 'full fill': 'thisFullFill',
                                        'partial fill': 'thisPartialFill', 'no fill': 'thisNoFill',
                                        'fillPerc': 'thisFillPerc', 'immedFillPerc': 'thisImmedFillPerc'})


prevResult = pd.read_csv(os.path.join(readPath, 'fillRate2_%s_%s.csv'%(prevStartDate, prevEndDate)))
# prevResult = prevResult.drop(columns=['# of orders'])
prevResult = prevResult.rename(columns={'% of orders': 'prevCountPerc', 'full fill': 'prevFullFill',
                                        'partial fill': 'prevPartialFill', 'no fill': 'prevNoFill',
                                        'fillPerc': 'prevFillPerc', 'immedFillPerc': 'prevImmedFillPerc'})

checkResult = pd.merge(thisResult, prevResult, how='left', on=['exchange', 'broker'], validate='one_to_one')
checkResult = checkResult.rename(columns={"# of orders_x": "this order", "# of orders_y": "prev order"})
checkResult['countPercDif'] = checkResult['thisCountPerc'] - checkResult['prevCountPerc']
checkResult['fullFillDif'] = checkResult['thisFullFill'] - checkResult['prevFullFill']
checkResult['partialFillDif'] = checkResult['thisPartialFill'] - checkResult['prevPartialFill']
checkResult['noFillDif'] = checkResult['thisNoFill'] - checkResult['prevNoFill']
checkResult['fillPercDif'] = checkResult['thisFillPerc'] - checkResult['prevFillPerc']
checkResult['immedFillPercDif'] = checkResult['thisImmedFillPerc'] - checkResult['prevImmedFillPerc']
re1 = checkResult[abs(checkResult['fullFillDif']) > 0.1]
re2 = checkResult[abs(checkResult['noFillDif']) > 0.1]
for col in ['prevFullFill', 'thisFullFill', 'fullFillDif',  'prevNoFill', 'thisNoFill', 'noFillDif']:
    re1[col] = re1[col].apply(lambda x: '%.0f%%'%(x*100))
    re2[col] = re2[col].apply(lambda x: '%.0f%%'%(x*100))

# display(checkResult[abs(checkResult['countPercDif']) > 0.1][['exchange', 'broker', 'prevCountPerc', 'thisCountPerc', 'countPercDif']])
# display(re1[['exchange', 'broker', 'prevFullFill', 'thisFullFill', 'fullFillDif']])
# display(checkResult[abs(checkResult['partialFillDif']) > 0.1][['exchange', 'broker', 'prevPartialFill', 'thisPartialFill', 'partialFillDif']])
# display(re2[['exchange', 'broker', 'prevNoFill', 'thisNoFill', 'noFillDif']])
# display(checkResult[abs(checkResult['fillPercDif']) > 0.1][['exchange', 'broker', 'prevFillPerc', 'thisFillPerc', 'fillPercDif']])
# display(checkResult[abs(checkResult['immedFillPercDif']) > 0.1][['exchange', 'broker', 'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']])
a = checkResult[((abs(checkResult['fillPercDif']) > 0.1))].reset_index(drop=True)
# exchangeTurnover, check trade notional in each broker
# a = pd.merge(a, exchangeTurnover.reset_index()[['exchange', 'broker', 'turnover']], how='left', on=['exchange', 'broker'], validate='one_to_one')
a["thisCount"] = a["this order"] * a["thisCountPerc"]
a["prevCount"] = a["prev order"] * a["prevCountPerc"]
a["thisCount"] = a["thisCount"].astype(int)
a["prevCount"] = a["prevCount"].astype(int) 
a = a[(a["thisCount"] > 100) & (a['prevCount'] > 100)]
a = a[['exchange', 'broker', 'thisCount','prevCount', 'prevFillPerc', 'thisFillPerc', 'fillPercDif',  'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']]
for col in ['prevFillPerc', 'thisFillPerc', 'fillPercDif',  'prevImmedFillPerc', 'thisImmedFillPerc', 'immedFillPercDif']:
#     a[col] = a[col].apply(lambda x: '%.0f%%'%(x*100))
    a[col] = a[col] * 100
a["thisCount"] = a["thisCount"].round(0).astype(int)
a["prevCount"] = a["prevCount"].round(0).astype(int)
display(a)
display(pd.concat([a[(a["thisCount"] > 100) & (a["prevCount"] > 100) & (a['exchange'] == 'SSE') & (a['broker'].str[:2] == 'zt')][['exchange', 'broker', 'prevFillPerc', 'thisFillPerc',  'fillPercDif', 'prevImmedFillPerc', 'thisImmedFillPerc']], \
                 a[(a["thisCount"] > 100) & (a["prevCount"] > 100) & (a['exchange'] == 'SZE') & (a['broker'].str[:2] == 'zs')][['exchange', 'broker', 'prevFillPerc', 'thisFillPerc',  'fillPercDif', 'prevImmedFillPerc', 'thisImmedFillPerc']]]).style.format({"prevFillPerc": "{:0,.0f}%", "thisFillPerc": "{:0,.0f}%","fillPercDif": "{:0,.0f}%","prevImmedFillPerc": "{:0,.0f}%",\
                  "thisImmedFillPerc": "{:0,.0f}%"}).applymap(color_negative_red, subset=['fillPercDif']))
# pd.concat([re1[['exchange', 'broker', 'prevFullFill', 'thisFullFill', 'fullFillDif','prevNoFill', 'thisNoFill', 'noFillDif']], 
#            re2[['exchange', 'broker', 'prevFullFill', 'thisFullFill', 'fullFillDif','prevNoFill', 'thisNoFill', 'noFillDif']]]).drop_duplicates()

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,exchange,broker,thisCount,prevCount,prevFillPerc,thisFillPerc,fillPercDif,prevImmedFillPerc,thisImmedFillPerc,immedFillPercDif
1,SZE,zs_88,4446,3544,73.322877,88.598028,15.275151,54.563327,80.581965,26.018639


,exchange,broker,prevFillPerc,thisFillPerc,fillPercDif,prevImmedFillPerc,thisImmedFillPerc
1,SZE,zs_88,73%,89%,15%,55%,81%
